In [ ]:
import pandas as pd
import gzip
from google.colab import drive
drive.mount('/content/drive')
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Function to read a compressed CSV file
def read_compressed_csv(file_path):
    with gzip.open(file_path, 'rb') as f:
        data = pd.read_csv(f)
    return data

In [ ]:
# Function to read larger files in chunks:
def read_compressed_csv_in_chunks(file_path, chunk_size=100000):
    chunks = []
    with gzip.open(file_path, 'rb') as f:
        reader = pd.read_csv(f, chunksize=chunk_size)
        for chunk in reader:
            chunks.append(chunk)
    return pd.concat(chunks, ignore_index=True)

In [ ]:
data_directory='/content/drive/MyDrive/mimic-iii-clinical-database-1.4'

## ADMISSIONS + PATIENTS

In [ ]:
admissions=pd.read_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/ADMISSIONS.csv')
print(admissions.columns)
print()
admissions = admissions[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'ADMISSION_TYPE',
                         'ADMISSION_LOCATION', 'RELIGION',
                         'ETHNICITY', 'DIAGNOSIS', 'HOSPITAL_EXPIRE_FLAG']]
admissions.head()

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME',
       'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION',
       'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION',
       'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS',
       'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA'],
      dtype='object')



,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,RELIGION,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG
0,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,EMERGENCY,EMERGENCY ROOM ADMIT,UNOBTAINABLE,WHITE,BENZODIAZEPINE OVERDOSE,0
1,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,CATHOLIC,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0
2,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,CATHOLIC,WHITE,BRAIN MASS,0
3,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,PROTESTANT QUAKER,WHITE,INTERIOR MYOCARDIAL INFARCTION,0
4,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,EMERGENCY,EMERGENCY ROOM ADMIT,UNOBTAINABLE,WHITE,ACUTE CORONARY SYNDROME,0


In [ ]:
patients=read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/PATIENTS.csv.gz')
print(patients.columns)
print()
patients = patients[['SUBJECT_ID', 'GENDER', 'DOB']]
patients.head()

Index(['ROW_ID', 'SUBJECT_ID', 'GENDER', 'DOB', 'DOD', 'DOD_HOSP', 'DOD_SSN',
       'EXPIRE_FLAG'],
      dtype='object')



,SUBJECT_ID,GENDER,DOB
0,249,F,2075-03-13 00:00:00
1,250,F,2164-12-27 00:00:00
2,251,M,2090-03-15 00:00:00
3,252,M,2078-03-06 00:00:00
4,253,F,2089-11-26 00:00:00


In [ ]:
merged_data = pd.merge(admissions, patients, how='inner', on=['SUBJECT_ID'])

In [ ]:
merged_data['AGE'] = merged_data.apply(lambda x: int(x['ADMITTIME'][:4]) - int(x['DOB'][:4]), axis=1)
merged_data['AGE'].value_counts().sort_index()

AGE
0      8089
1        21
15        8
16       11
17       51
       ... 
307      22
308      15
309       5
310       7
311       2
Name: count, Length: 89, dtype: int64

In [ ]:
merged_data['AGE'] = merged_data.apply(lambda x: x['AGE']-300+90 if x['AGE'] >= 300 else x['AGE'], axis=1)
merged_data = merged_data[merged_data['AGE']>=16]

In [ ]:
merged_data['ADMITTIME'] = pd.to_datetime(merged_data['ADMITTIME'])
merged_data['DISCHTIME'] = pd.to_datetime(merged_data['DISCHTIME'])

<ipython-input-205-d68f3a601461>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data['ADMITTIME'] = pd.to_datetime(merged_data['ADMITTIME'])


In [ ]:
# Some rows have incorrect entries as follows
merged_data[merged_data['DISCHTIME'] - merged_data['ADMITTIME'] < pd.Timedelta(days=0)]

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,RELIGION,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,GENDER,DOB,AGE
426,417,102633,2177-03-23 16:17:00,2177-03-23 07:20:00,URGENT,PHYS REFERRAL/NORMAL DELI,UNOBTAINABLE,WHITE,ORGAN DONOR ACCOUNT,1,F,2126-12-27 00:00:00,51
457,181,102631,2153-10-12 09:49:00,2153-10-12 06:29:00,EMERGENCY,EMERGENCY ROOM ADMIT,PROTESTANT QUAKER,WHITE,DISSECTING ANEURYSIM,1,M,2088-07-24 00:00:00,65
696,516,187482,2197-07-31 20:18:00,2197-07-31 01:10:00,EMERGENCY,EMERGENCY ROOM ADMIT,JEWISH,UNKNOWN/NOT SPECIFIED,RESPIRATORY DISTRESS,1,F,2126-08-07 00:00:00,71
1974,1381,181430,2189-01-02 14:25:00,2189-01-02 12:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,UNOBTAINABLE,WHITE,STROKE;TELEMETRY,1,F,2107-07-15 00:00:00,82
2785,2420,135098,2184-12-01 19:28:00,2184-12-01 16:50:00,EMERGENCY,EMERGENCY ROOM ADMIT,CATHOLIC,WHITE,ALTERED MENTAL STATUS,0,F,2136-05-20 00:00:00,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57240,96810,165589,2103-09-25 20:38:00,2103-09-25 12:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,UNOBTAINABLE,WHITE,CARDIAC ARREST,1,F,1803-07-11 00:00:00,90
57567,98943,193747,2164-11-14 20:03:00,2164-11-14 12:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,UNOBTAINABLE,WHITE,INTRACRANIAL HEMORRHAGE,1,M,2083-12-04 00:00:00,81
57911,99207,191004,2143-07-06 19:59:00,2143-07-06 12:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,CATHOLIC,WHITE,GSW L. CHEST WALL & ABDOMEN,1,M,2100-04-23 00:00:00,43
58356,90642,101946,2122-04-24 14:36:00,2122-04-24 12:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,CATHOLIC,UNABLE TO OBTAIN,ST-SEGMENT ELEVATION MYOCARDIAL INFARCTION\CATH,1,M,2049-07-17 00:00:00,73


In [ ]:
# Getting rid of the incorrect entries as above
merged_data = merged_data[merged_data['DISCHTIME'] - merged_data['ADMITTIME'] > pd.Timedelta(days=0)]

In [ ]:
# # For training the model we cannot use the data of entire duration of hospital stay of a patient because during inference, we would be running the model at some intermediate point.
# # Hence, taking the initial 30% of entire stay data of a patient, and calling the 30% datetimestamp as RUNTIME
# merged_data['RUNTIME'] = merged_data['ADMITTIME'] + 0.3 * (merged_data['DISCHTIME'] - merged_data['ADMITTIME'])

In [ ]:
# Creating a RUNTIME variable which is = ADMITTIME + 2 days, since we will be running the model at inference after days of admission for a patient
# Hence dropping all data which has duration of stay < 2 days
merged_data = merged_data[merged_data['DISCHTIME'] - merged_data['ADMITTIME'] > pd.Timedelta(days=2)]
merged_data['RUNTIME'] = merged_data['ADMITTIME'] + pd.Timedelta(days=2)

In [ ]:
merged_data = merged_data.drop(['ADMITTIME', 'DISCHTIME', 'DOB'], axis=1)

In [ ]:
merged_data['RELIGION'].unique()

array(['CATHOLIC', 'PROTESTANT QUAKER', 'UNOBTAINABLE', 'NOT SPECIFIED',
       'JEWISH', 'BUDDHIST', "JEHOVAH'S WITNESS", 'OTHER', nan,
       'GREEK ORTHODOX', 'EPISCOPALIAN', 'CHRISTIAN SCIENTIST', 'HEBREW',
       'METHODIST', 'UNITARIAN-UNIVERSALIST', 'HINDU', 'BAPTIST',
       '7TH DAY ADVENTIST', 'MUSLIM', 'ROMANIAN EAST. ORTH', 'LUTHERAN'],
      dtype=object)

In [ ]:
merged_data['RELIGION'] = merged_data['RELIGION'].fillna("UNOBTAINABLE")

In [ ]:
'UNKNOWN' in list(merged_data['DIAGNOSIS'].unique())

False

In [ ]:
merged_data['DIAGNOSIS'] = merged_data['DIAGNOSIS'].fillna("'UNKNOWN'")

In [ ]:
merged_data.isnull().sum()

SUBJECT_ID              0
HADM_ID                 0
ADMISSION_TYPE          0
ADMISSION_LOCATION      0
RELIGION                0
ETHNICITY               0
DIAGNOSIS               0
HOSPITAL_EXPIRE_FLAG    0
GENDER                  0
AGE                     0
RUNTIME                 0
dtype: int64

In [ ]:
print(len(merged_data))
merged_data.head()

46188


,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,ADMISSION_LOCATION,RELIGION,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,GENDER,AGE,RUNTIME
1,23,152223,ELECTIVE,PHYS REFERRAL/NORMAL DELI,CATHOLIC,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,M,71,2153-09-05 07:15:00
2,23,124321,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,CATHOLIC,WHITE,BRAIN MASS,0,M,75,2157-10-20 19:34:00
3,24,161859,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,PROTESTANT QUAKER,WHITE,INTERIOR MYOCARDIAL INFARCTION,0,M,39,2139-06-08 16:14:00
4,25,129635,EMERGENCY,EMERGENCY ROOM ADMIT,UNOBTAINABLE,WHITE,ACUTE CORONARY SYNDROME,0,M,59,2160-11-04 02:06:00
5,26,197661,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,CATHOLIC,UNKNOWN/NOT SPECIFIED,V-TACH,0,M,72,2126-05-08 15:16:00


In [ ]:
# merged_data.to_csv("/content/drive/MyDrive/mimic-iii-clinical-database-1.4/Merged_files/adm_pat_v3.csv", index=False)

## CPTEVENTS

In [ ]:
cptevents=read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/CPTEVENTS.csv.gz')
len(cptevents), cptevents.nunique()

<ipython-input-101-27d2bf21da2b>:4: DtypeWarning: Columns (4,5,7,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(f)


(573146,
 ROW_ID              573146
 SUBJECT_ID           34005
 HADM_ID              44148
 COSTCENTER               2
 CHARTDATE            34779
 CPT_CD                3060
 CPT_NUMBER            2012
 CPT_SUFFIX               1
 TICKET_ID_SEQ          900
 SECTIONHEADER            7
 SUBSECTIONHEADER        54
 DESCRIPTION              4
 dtype: int64)

In [ ]:
len(cptevents), cptevents.isnull().sum()

(573146,
 ROW_ID                   0
 SUBJECT_ID               0
 HADM_ID                  0
 COSTCENTER               0
 CHARTDATE           471601
 CPT_CD                   0
 CPT_NUMBER              18
 CPT_SUFFIX          573124
 TICKET_ID_SEQ       101545
 SECTIONHEADER           21
 SUBSECTIONHEADER        21
 DESCRIPTION         471601
 dtype: int64)

In [ ]:
cptevents = cptevents.sort_values(by=['HADM_ID', 'TICKET_ID_SEQ'])

In [ ]:
count, id_prev, id_cur = 1, 0, 0
def counter_func(x):
  global count, id_prev, id_cur
  count += 1
  id_cur = x['HADM_ID']
  if id_prev != id_cur:
    id_prev = id_cur
    count = 1
  return count

cptevents['TICKET_ID_SEQ'] = cptevents.apply(counter_func, axis=1)

In [ ]:
assert all(cptevents['HADM_ID'].value_counts().sort_index() == cptevents.groupby('HADM_ID')['TICKET_ID_SEQ'].max() - cptevents.groupby('HADM_ID')['TICKET_ID_SEQ'].min() + 1)
cptevents.head(17)

,ROW_ID,SUBJECT_ID,HADM_ID,COSTCENTER,CHARTDATE,CPT_CD,CPT_NUMBER,CPT_SUFFIX,TICKET_ID_SEQ,SECTIONHEADER,SUBSECTIONHEADER,DESCRIPTION
407988,407416,58526,100001,ICU,NaN,99291,99291.0,NaN,1,Evaluation and management,Critical care services,NaN
407989,407417,58526,100001,ICU,NaN,99291,99291.0,NaN,2,Evaluation and management,Critical care services,NaN
407990,407418,58526,100001,ICU,NaN,99291,99291.0,NaN,3,Evaluation and management,Critical care services,NaN
407991,407419,58526,100001,ICU,NaN,99291,99291.0,NaN,4,Evaluation and management,Critical care services,NaN
407992,407420,58526,100001,ICU,NaN,99233,99233.0,NaN,5,Evaluation and management,Hospital inpatient services,NaN
407993,407421,58526,100001,ICU,NaN,99233,99233.0,NaN,6,Evaluation and management,Hospital inpatient services,NaN
407986,407414,58526,100001,ICU,NaN,99232,99232.0,NaN,7,Evaluation and management,Hospital inpatient services,NaN
407987,407415,58526,100001,ICU,NaN,99239,99239.0,NaN,8,Evaluation and management,Hospital inpatient services,NaN
241868,243108,54610,100003,ICU,NaN,99291,99291.0,NaN,1,Evaluation and management,Critical care services,NaN
241869,243109,54610,100003,ICU,NaN,36556,36556.0,NaN,2,Surgery,Cardiovascular system,NaN


In [ ]:
cptevents['SUBSECTIONHEADER'] = cptevents['SUBSECTIONHEADER'].fillna("Other procedures")

In [ ]:
cptevents.isna().sum()

ROW_ID                   0
SUBJECT_ID               0
HADM_ID                  0
COSTCENTER               0
CHARTDATE           471601
CPT_CD                   0
CPT_NUMBER              18
CPT_SUFFIX          573124
TICKET_ID_SEQ            0
SECTIONHEADER           21
SUBSECTIONHEADER         0
DESCRIPTION         471601
dtype: int64

In [ ]:
cptevents = cptevents[['SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CPT_CD', 'TICKET_ID_SEQ', 'SUBSECTIONHEADER']]

In [ ]:
cptevents['CHARTDATE'].fillna('2262-04-11 00:00:00', inplace=True)      # This date is the max. possible date allowed by pandas datetime/timestamp
cptevents['CHARTDATE'] = pd.to_datetime(cptevents['CHARTDATE'])

<ipython-input-226-7c506d40d725>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cptevents['CHARTDATE'].fillna('2262-04-11 00:00:00', inplace=True)      # This date is the max. possible date allowed by pandas datetime/timestamp
<ipython-input-226-7c506d40d725>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cptevents['CHARTDATE'] = pd.to_datetime(cptevents['CHARTDATE'])


In [ ]:
cptevents['HADM_ID'].nunique()

44148

In [ ]:
merged_data = pd.merge(merged_data, cptevents, how='inner', on=['SUBJECT_ID', 'HADM_ID'])

In [ ]:
## We need to take those ticket id seq which have chartdate less than runtime and are consecutive starting from 1 to some max. value
max_ticket_id_df = merged_data[merged_data['CHARTDATE'] < merged_data['RUNTIME']].groupby('HADM_ID')['TICKET_ID_SEQ'].max().reset_index()
max_ticket_id_df

,HADM_ID,TICKET_ID_SEQ
0,100009,11
1,100011,37
2,100012,11
3,100016,15
4,100018,19
...,...,...
14073,199969,2
14074,199972,9
14075,199976,13
14076,199979,7


In [ ]:
merged_data = pd.merge(merged_data, max_ticket_id_df, how='inner', on='HADM_ID')
merged_data = merged_data[merged_data['TICKET_ID_SEQ_x'] <= merged_data['TICKET_ID_SEQ_y']]

In [ ]:
merged_data.drop(columns=['TICKET_ID_SEQ_y', 'CHARTDATE'], inplace=True)
merged_data.rename(columns={'TICKET_ID_SEQ_x': 'TICKET_ID_SEQ', 'SUBSECTIONHEADER': 'PROCEDURE'}, inplace=True)

In [ ]:
del max_ticket_id_df

In [ ]:
# merged_data.to_csv("/content/drive/MyDrive/mimic-iii-clinical-database-1.4/Merged_files/adm_pat_cpt_v3.csv", index=False)

## CALLOUT (Ignore)

In [ ]:
callout = read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/CALLOUT.csv.gz')
len(callout), callout.nunique()

(34499,
 ROW_ID                    34499
 SUBJECT_ID                22871
 HADM_ID                   28732
 SUBMIT_WARDID                 9
 SUBMIT_CAREUNIT               5
 CURR_WARDID                  35
 CURR_CAREUNIT                 5
 CALLOUT_WARDID               37
 CALLOUT_SERVICE              21
 REQUEST_TELE                  2
 REQUEST_RESP                  2
 REQUEST_CDIFF                 2
 REQUEST_MRSA                  2
 REQUEST_VRE                   2
 CALLOUT_STATUS                1
 CALLOUT_OUTCOME               2
 DISCHARGE_WARDID             27
 ACKNOWLEDGE_STATUS            4
 CREATETIME                34497
 UPDATETIME                34498
 ACKNOWLEDGETIME           32706
 OUTCOMETIME               34479
 FIRSTRESERVATIONTIME      15255
 CURRENTRESERVATIONTIME     1164
 dtype: int64)

In [ ]:
callout.isnull().sum()

ROW_ID                        0
SUBJECT_ID                    0
HADM_ID                       0
SUBMIT_WARDID                 1
SUBMIT_CAREUNIT           29377
CURR_WARDID                   1
CURR_CAREUNIT                 1
CALLOUT_WARDID                0
CALLOUT_SERVICE               0
REQUEST_TELE                  0
REQUEST_RESP                  0
REQUEST_CDIFF                 0
REQUEST_MRSA                  0
REQUEST_VRE                   0
CALLOUT_STATUS                0
CALLOUT_OUTCOME               0
DISCHARGE_WARDID           4532
ACKNOWLEDGE_STATUS            0
CREATETIME                    0
UPDATETIME                    0
ACKNOWLEDGETIME            1792
OUTCOMETIME                   0
FIRSTRESERVATIONTIME      19239
CURRENTRESERVATIONTIME    33335
dtype: int64

In [ ]:
callout

,ROW_ID,SUBJECT_ID,HADM_ID,SUBMIT_WARDID,SUBMIT_CAREUNIT,CURR_WARDID,CURR_CAREUNIT,CALLOUT_WARDID,CALLOUT_SERVICE,REQUEST_TELE,...,CALLOUT_STATUS,CALLOUT_OUTCOME,DISCHARGE_WARDID,ACKNOWLEDGE_STATUS,CREATETIME,UPDATETIME,ACKNOWLEDGETIME,OUTCOMETIME,FIRSTRESERVATIONTIME,CURRENTRESERVATIONTIME
0,402,854,175684,52.0,NaN,29.0,MICU,1,MED,0,...,Inactive,Discharged,29.0,Acknowledged,2146-10-05 13:16:55,2146-10-05 13:16:55,2146-10-05 13:24:00,2146-10-05 18:55:22,2146-10-05 15:27:44,NaN
1,403,864,138624,15.0,NaN,55.0,CSRU,55,CSURG,0,...,Inactive,Discharged,55.0,Acknowledged,2114-11-28 08:31:39,2114-11-28 09:42:08,2114-11-28 09:43:08,2114-11-28 12:10:02,NaN,NaN
2,404,864,138624,12.0,NaN,55.0,CSRU,55,CSURG,1,...,Inactive,Discharged,55.0,Acknowledged,2114-11-30 10:24:25,2114-12-01 09:06:18,2114-12-01 12:26:05,2114-12-01 21:55:05,NaN,NaN
3,405,867,184298,7.0,NaN,17.0,CCU,17,CCU,1,...,Inactive,Discharged,17.0,Acknowledged,2136-12-29 08:45:42,2136-12-29 10:17:16,2136-12-29 10:33:51,2136-12-29 18:10:02,NaN,NaN
4,157,306,167129,57.0,NaN,3.0,SICU,44,NSURG,1,...,Inactive,Discharged,3.0,Acknowledged,2199-09-18 11:47:47,2199-09-18 11:47:47,2199-09-18 11:58:33,2199-09-18 15:10:02,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34494,32674,94046,199742,23.0,NaN,54.0,MICU,1,MED,1,...,Inactive,Discharged,54.0,Acknowledged,2112-11-01 11:13:04,2112-11-01 11:13:04,2112-11-01 11:16:14,2112-11-01 19:42:07,2112-11-01 16:12:28,NaN
34495,32675,94049,135809,23.0,NaN,45.0,MICU,1,MED,0,...,Inactive,Discharged,45.0,Acknowledged,2112-10-09 14:12:09,2112-10-09 14:12:09,2112-10-09 14:23:52,2112-10-09 17:40:05,NaN,NaN
34496,32676,94050,173013,23.0,NaN,45.0,MICU,1,MED,0,...,Inactive,Discharged,45.0,Acknowledged,2165-10-07 10:18:42,2165-10-07 10:18:42,2165-10-07 10:20:31,2165-10-07 19:10:11,2165-10-07 17:10:10,NaN
34497,32677,94056,172374,50.0,NaN,45.0,MICU,1,MED,0,...,Inactive,Discharged,45.0,Acknowledged,2199-10-30 15:35:30,2199-10-30 15:35:30,2199-10-30 16:16:09,2199-10-30 22:10:04,NaN,NaN


## CAREGIVERS (Ignore)

In [ ]:
caregivers = pd.read_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/CAREGIVERS.csv')
len(caregivers), caregivers.nunique()

(7567,
 ROW_ID         7567
 CGID           7567
 LABEL           230
 DESCRIPTION      16
 dtype: int64)

In [ ]:
caregivers.isnull().sum()

ROW_ID            0
CGID              0
LABEL            49
DESCRIPTION    2412
dtype: int64

In [ ]:
caregivers

,ROW_ID,CGID,LABEL,DESCRIPTION
0,2228,16174,RO,Read Only
1,2229,16175,RO,Read Only
2,2230,16176,Res,Resident/Fellow/PA/NP
3,2231,16177,RO,Read Only
4,2232,16178,RT,Respiratory
...,...,...,...,...
7562,6300,20303,MD,NaN
7563,6301,20304,RN,RN
7564,6302,20305,MDs,NaN
7565,6303,20306,RPH,Pharmacist


## CHARTEVENTS (Ignore) (Large)

In [ ]:
# chartevents = read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/CHARTEVENTS.csv.gz')
# len(chartevents), chartevents.nunique()

In [ ]:
# chartevents.isnull().sum()

In [ ]:
# chartevents

## D_CPT (Ignore)

In [ ]:
d_cpt=read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/D_CPT.csv.gz')
len(d_cpt), d_cpt.nunique()

(134,
 ROW_ID                 134
 CATEGORY                 3
 SECTIONRANGE             9
 SECTIONHEADER            8
 SUBSECTIONRANGE        134
 SUBSECTIONHEADER       132
 CODESUFFIX               2
 MINCODEINSUBSECTION    133
 MAXCODEINSUBSECTION    134
 dtype: int64)

In [ ]:
d_cpt.isnull().sum()

ROW_ID                   0
CATEGORY                 0
SECTIONRANGE             0
SECTIONHEADER            0
SUBSECTIONRANGE          0
SUBSECTIONHEADER         0
CODESUFFIX             123
MINCODEINSUBSECTION      0
MAXCODEINSUBSECTION      0
dtype: int64

In [ ]:
d_cpt

,ROW_ID,CATEGORY,SECTIONRANGE,SECTIONHEADER,SUBSECTIONRANGE,SUBSECTIONHEADER,CODESUFFIX,MINCODEINSUBSECTION,MAXCODEINSUBSECTION
0,1,1,99201-99499,Evaluation and management,99201-99216,Office/other outpatient services,NaN,99201,99216
1,2,1,99201-99499,Evaluation and management,99217-99220,Hospital observation services,NaN,99217,99220
2,3,1,99201-99499,Evaluation and management,99221-99239,Hospital inpatient services,NaN,99221,99239
3,4,1,99201-99499,Evaluation and management,99241-99255,Consultations,NaN,99241,99255
4,5,1,99201-99499,Evaluation and management,99261-99263,Follow-up inpatient consultations (deleted codes),NaN,99261,99263
...,...,...,...,...,...,...,...,...,...
129,130,2,0001F-7025F,Performance measurement,5005F-5100F,Follow-up or other outcomes,F,5005,5100
130,131,2,0001F-7025F,Performance measurement,6005F-6045F,Patient safety,F,6005,6045
131,132,2,0001F-7025F,Performance measurement,7010F-7025F,Structural Measures,F,7010,7025
132,133,3,0016T-0207T,Emerging technology,0016T-0207T,Temporary codes,T,16,207


## D_ICD_DIAGNOSES

In [ ]:
d_icd_diagnoses = read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/D_ICD_DIAGNOSES.csv.gz')
len(d_icd_diagnoses), d_icd_diagnoses.nunique()

(14567,
 ROW_ID         14567
 ICD9_CODE      14567
 SHORT_TITLE    14328
 LONG_TITLE     14562
 dtype: int64)

In [ ]:
d_icd_diagnoses.isnull().sum()

ROW_ID         0
ICD9_CODE      0
SHORT_TITLE    0
LONG_TITLE     0
dtype: int64

In [ ]:
d_icd_diagnoses

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,174,01166,TB pneumonia-oth test,"Tuberculous pneumonia [any form], tubercle bac..."
1,175,01170,TB pneumothorax-unspec,"Tuberculous pneumothorax, unspecified"
2,176,01171,TB pneumothorax-no exam,"Tuberculous pneumothorax, bacteriological or h..."
3,177,01172,TB pneumothorx-exam unkn,"Tuberculous pneumothorax, bacteriological or h..."
4,178,01173,TB pneumothorax-micro dx,"Tuberculous pneumothorax, tubercle bacilli fou..."
...,...,...,...,...
14562,14432,V7399,Scrn unspcf viral dis,Special screening examination for unspecified ...
14563,14433,V740,Screening for cholera,Screening examination for cholera
14564,14434,V741,Screening-pulmonary TB,Screening examination for pulmonary tuberculosis
14565,14435,V742,Screening for leprosy,Screening examination for leprosy (Hansen's di...


## D_ICD_PROCEDURES

In [ ]:
d_icd_procedures = read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/D_ICD_PROCEDURES.csv.gz')
len(d_icd_procedures), d_icd_procedures.nunique()

(3882,
 ROW_ID         3882
 ICD9_CODE      3809
 SHORT_TITLE    3882
 LONG_TITLE     3882
 dtype: int64)

In [ ]:
d_icd_procedures.isnull().sum()

ROW_ID         0
ICD9_CODE      0
SHORT_TITLE    0
LONG_TITLE     0
dtype: int64

In [ ]:
d_icd_procedures

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,264,851,Canthotomy,Canthotomy
1,265,852,Blepharorrhaphy,Blepharorrhaphy
2,266,859,Adjust lid position NEC,Other adjustment of lid position
3,267,861,Lid reconst w skin graft,Reconstruction of eyelid with skin flap or graft
4,268,862,Lid reconst w muc graft,Reconstruction of eyelid with mucous membrane ...
...,...,...,...,...
3877,3344,9959,Vaccination/innocula NEC,Other vaccination and inoculation
3878,3345,9960,Cardiopulm resuscita NOS,"Cardiopulmonary resuscitation, not otherwise s..."
3879,3346,9961,Atrial cardioversion,Atrial cardioversion
3880,3347,9962,Heart countershock NEC,Other electric countershock of heart


## D_ITEMS (Ignore)

In [ ]:
d_items = read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/D_ITEMS.csv.gz')
len(d_items), d_items.nunique()

(12487,
 ROW_ID          12487
 ITEMID          12487
 LABEL           11846
 ABBREVIATION     2907
 DBSOURCE            3
 LINKSTO             7
 CATEGORY           94
 UNITNAME           53
 PARAM_TYPE          7
 CONCEPTID           0
 dtype: int64)

In [ ]:
d_items.isnull().sum()

ROW_ID              0
ITEMID              0
LABEL               4
ABBREVIATION     9495
DBSOURCE            0
LINKSTO             0
CATEGORY         6438
UNITNAME        11253
PARAM_TYPE       9495
CONCEPTID       12487
dtype: int64

In [ ]:
d_items

,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
0,457,497,Patient controlled analgesia (PCA) [Inject],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1,458,498,PCA Lockout (Min),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2,459,499,PCA Medication,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
3,460,500,PCA Total Dose,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4,461,501,PCV Exh Vt (Obser),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
12482,14518,226757,GCSMotorApacheIIValue,GCSMotorApacheIIValue,metavision,chartevents,Scores - APACHE II,NaN,Text,NaN
12483,14519,226758,GCSVerbalApacheIIValue,GCSVerbalApacheIIValue,metavision,chartevents,Scores - APACHE II,NaN,Text,NaN
12484,14520,226759,HCO3ApacheIIValue,HCO3ApacheIIValue,metavision,chartevents,Scores - APACHE II,None,Numeric,NaN
12485,14521,226760,HCO3Score,HCO3Score,metavision,chartevents,Scores - APACHE II,None,Numeric,NaN


## D_LABITEMS (Ignore)

In [ ]:
d_labitems = read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/D_LABITEMS.csv.gz')
len(d_labitems), d_labitems.nunique()

(753,
 ROW_ID        753
 ITEMID        753
 LABEL         589
 FLUID          16
 CATEGORY        6
 LOINC_CODE    575
 dtype: int64)

In [ ]:
d_labitems.isnull().sum()

ROW_ID          0
ITEMID          0
LABEL           0
FLUID           0
CATEGORY        0
LOINC_CODE    168
dtype: int64

In [ ]:
d_labitems

,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
0,546,51346,Blasts,Cerebrospinal Fluid (CSF),Hematology,26447-3
1,547,51347,Eosinophils,Cerebrospinal Fluid (CSF),Hematology,26451-5
2,548,51348,"Hematocrit, CSF",Cerebrospinal Fluid (CSF),Hematology,30398-2
3,549,51349,Hypersegmented Neutrophils,Cerebrospinal Fluid (CSF),Hematology,26506-6
4,550,51350,Immunophenotyping,Cerebrospinal Fluid (CSF),Hematology,NaN
...,...,...,...,...,...,...
748,749,51551,VOIDED SPECIMEN,OTHER BODY FLUID,HEMATOLOGY,NaN
749,750,51552,VOIDED SPECIMEN,STOOL,CHEMISTRY,NaN
750,751,51553,VOIDED SPECIMEN,URINE,CHEMISTRY,NaN
751,752,51554,VOIDED SPECIMEN,JOINT FLUID,HEMATOLOGY,NaN


## DATETIMEEVENTS (Ignore) (Large)

In [ ]:
# datetimeevents = read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/DATETIMEEVENTS.csv.gz')
# len(datetimeevents), datetimeevents.nunique()

In [ ]:
# datetimeevents.isnull().sum()

In [ ]:
# datetimeevents()

## DIAGNOSES_ICD

In [ ]:
diagnoses_icd = read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/DIAGNOSES_ICD.csv.gz')
len(diagnoses_icd), diagnoses_icd.nunique()

(651047,
 ROW_ID        651047
 SUBJECT_ID     46520
 HADM_ID        58976
 SEQ_NUM           39
 ICD9_CODE       6984
 dtype: int64)

In [ ]:
diagnoses_icd.isnull().sum()

ROW_ID         0
SUBJECT_ID     0
HADM_ID        0
SEQ_NUM       47
ICD9_CODE     47
dtype: int64

In [ ]:
diagnoses_icd

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254
...,...,...,...,...,...
651042,639798,97503,188195,2.0,20280
651043,639799,97503,188195,3.0,V5869
651044,639800,97503,188195,4.0,V1279
651045,639801,97503,188195,5.0,5275


## DRGCODES

In [ ]:
drgcodes = read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/DRGCODES.csv.gz')
len(drgcodes), drgcodes.nunique()

(125557,
 ROW_ID           125557
 SUBJECT_ID        46511
 HADM_ID           58890
 DRG_TYPE              3
 DRG_CODE           1667
 DESCRIPTION        1367
 DRG_SEVERITY          5
 DRG_MORTALITY         5
 dtype: int64)

In [ ]:
drgcodes.isnull().sum()

ROW_ID               0
SUBJECT_ID           0
HADM_ID              0
DRG_TYPE             0
DRG_CODE             0
DESCRIPTION         63
DRG_SEVERITY     58923
DRG_MORTALITY    58923
dtype: int64

In [ ]:
drgcodes

,ROW_ID,SUBJECT_ID,HADM_ID,DRG_TYPE,DRG_CODE,DESCRIPTION,DRG_SEVERITY,DRG_MORTALITY
0,342,2491,144486,HCFA,28,"TRAUMATIC STUPOR & COMA, COMA <1 HR AGE >17 WI...",NaN,NaN
1,343,24958,162910,HCFA,110,MAJOR CARDIOVASCULAR PROCEDURES WITH COMPLICAT...,NaN,NaN
2,344,18325,153751,HCFA,390,NEONATE WITH OTHER SIGNIFICANT PROBLEMS,NaN,NaN
3,345,17887,182692,HCFA,14,SPECIFIC CEREBROVASCULAR DISORDERS EXCEPT TRAN...,NaN,NaN
4,346,11113,157980,HCFA,390,NEONATE WITH OTHER SIGNIFICANT PROBLEMS,NaN,NaN
...,...,...,...,...,...,...,...,...
125552,123452,71582,101422,MS,221,CARDIAC VALVE & OTH MAJ CARDIOTHORACIC PROC W/...,NaN,NaN
125553,123453,46449,110075,APR,1653,Coronary Bypass w/ Cardiac Cath Or Percutaneou...,3.0,2.0
125554,123454,46449,110075,APR,1653,Coronary Bypass w/ Cardiac Cath Or Percutaneou...,3.0,2.0
125555,123455,46449,110075,MS,234,CORONARY BYPASS W CARDIAC CATH W/O MCC,NaN,NaN


In [ ]:
x = list(drgcodes['DESCRIPTION'].unique())
# for i in range(len(x)):
#     if 'w' in x[i] or 'CC' in

## ICUSTAYS

In [ ]:
merged_data['HADM_ID'].nunique()

14078

In [ ]:
icustays = pd.read_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/ICUSTAYS.csv')
len(icustays), icustays.nunique()

(61532,
 ROW_ID            61532
 SUBJECT_ID        46476
 HADM_ID           57786
 ICUSTAY_ID        61532
 DBSOURCE              3
 FIRST_CAREUNIT        6
 LAST_CAREUNIT         6
 FIRST_WARDID         16
 LAST_WARDID          17
 INTIME            61531
 OUTTIME           61518
 LOS               40175
 dtype: int64)

In [ ]:
icustays['LOS'].describe()

count    61522.000000
mean         4.917972
std          9.638784
min          0.000100
25%          1.108025
50%          2.092250
75%          4.483175
max        173.072500
Name: LOS, dtype: float64

In [ ]:
icustays.isnull().sum()

ROW_ID             0
SUBJECT_ID         0
HADM_ID            0
ICUSTAY_ID         0
DBSOURCE           0
FIRST_CAREUNIT     0
LAST_CAREUNIT      0
FIRST_WARDID       0
LAST_WARDID        0
INTIME             0
OUTTIME           10
LOS               10
dtype: int64

In [ ]:
icustays

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202
...,...,...,...,...,...,...,...,...,...,...,...,...
61527,59806,94944,143774,201233,metavision,CSRU,CSRU,15,15,2104-04-15 10:18:16,2104-04-17 14:51:00,2.1894
61528,59807,94950,123750,283653,metavision,CCU,CCU,7,7,2155-12-08 05:33:16,2155-12-10 17:24:58,2.4942
61529,59808,94953,196881,241585,metavision,SICU,SICU,57,57,2160-03-03 16:09:11,2160-03-04 14:22:33,0.9259
61530,59809,94954,118475,202802,metavision,CSRU,CSRU,15,15,2183-03-25 09:53:10,2183-03-27 17:55:03,2.3346


In [ ]:
icustays = icustays[['SUBJECT_ID', 'HADM_ID', 'INTIME', 'OUTTIME', 'LOS']]
icustays.dropna(inplace=True)       # The NA values belong to the SUBJECT_IDs that we have already dropped while merging the data earlier

<ipython-input-239-d8624d8859a2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  icustays.dropna(inplace=True)       # The NA values belong to the SUBJECT_IDs that we have already dropped while merging the data earlier


In [ ]:
icustays

,SUBJECT_ID,HADM_ID,INTIME,OUTTIME,LOS
0,268,110404,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,269,106296,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,270,188028,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,271,173727,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,272,164716,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202
...,...,...,...,...,...
61527,94944,143774,2104-04-15 10:18:16,2104-04-17 14:51:00,2.1894
61528,94950,123750,2155-12-08 05:33:16,2155-12-10 17:24:58,2.4942
61529,94953,196881,2160-03-03 16:09:11,2160-03-04 14:22:33,0.9259
61530,94954,118475,2183-03-25 09:53:10,2183-03-27 17:55:03,2.3346


In [ ]:
merged_data = pd.merge(merged_data, icustays, how='inner', on=['SUBJECT_ID', 'HADM_ID'])

In [ ]:
merged_data['INTIME'] = pd.to_datetime(merged_data['INTIME'])
merged_data['OUTTIME'] = pd.to_datetime(merged_data['OUTTIME'])

In [ ]:
merged_data

,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,ADMISSION_LOCATION,RELIGION,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,GENDER,AGE,RUNTIME,CPT_CD,TICKET_ID_SEQ,PROCEDURE,INTIME,OUTTIME,LOS
0,31,128652,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,CATHOLIC,WHITE,STATUS EPILEPTICUS,1,M,72,2108-08-24 23:27:00,99251,1,Consultations,2108-08-22 23:28:42,2108-08-30 21:59:20,7.9379
1,31,128652,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,CATHOLIC,WHITE,STATUS EPILEPTICUS,1,M,72,2108-08-24 23:27:00,94003,2,Pulmonary,2108-08-22 23:28:42,2108-08-30 21:59:20,7.9379
2,31,128652,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,CATHOLIC,WHITE,STATUS EPILEPTICUS,1,M,72,2108-08-24 23:27:00,94003,3,Pulmonary,2108-08-22 23:28:42,2108-08-30 21:59:20,7.9379
3,36,165660,ELECTIVE,PHYS REFERRAL/NORMAL DELI,NOT SPECIFIED,WHITE,VENTRAL HERNIA/SDA,0,M,73,2134-05-12 11:30:00,99222,1,Hospital inpatient services,2134-05-12 06:52:47,2134-05-16 15:14:20,4.3483
4,36,165660,ELECTIVE,PHYS REFERRAL/NORMAL DELI,NOT SPECIFIED,WHITE,VENTRAL HERNIA/SDA,0,M,73,2134-05-12 11:30:00,99291,2,Critical care services,2134-05-12 06:52:47,2134-05-16 15:14:20,4.3483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246447,98805,122631,ELECTIVE,PHYS REFERRAL/NORMAL DELI,NOT SPECIFIED,WHITE,RENAL CANCER/SDA,0,M,42,2200-09-14 07:15:00,99291,2,Critical care services,2200-09-12 19:25:51,2200-09-14 18:17:01,1.9522
246448,98805,122631,ELECTIVE,PHYS REFERRAL/NORMAL DELI,NOT SPECIFIED,WHITE,RENAL CANCER/SDA,0,M,42,2200-09-14 07:15:00,99233,3,Hospital inpatient services,2200-09-12 19:25:51,2200-09-14 18:17:01,1.9522
246449,98805,122631,ELECTIVE,PHYS REFERRAL/NORMAL DELI,NOT SPECIFIED,WHITE,RENAL CANCER/SDA,0,M,42,2200-09-14 07:15:00,44140,4,Digestive system,2200-09-12 19:25:51,2200-09-14 18:17:01,1.9522
246450,98805,122631,ELECTIVE,PHYS REFERRAL/NORMAL DELI,NOT SPECIFIED,WHITE,RENAL CANCER/SDA,0,M,42,2200-09-14 07:15:00,44955,5,Digestive system,2200-09-12 19:25:51,2200-09-14 18:17:01,1.9522


In [ ]:
merged_data = merged_data[merged_data['INTIME'] < merged_data['RUNTIME']]

In [ ]:
merged_data['LOS_ACTUAL'] = merged_data['OUTTIME'] - merged_data['INTIME']

<ipython-input-245-545feef4af33>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data['LOS_ACTUAL'] = merged_data['OUTTIME'] - merged_data['INTIME']


In [ ]:
merged_data.head()

,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,ADMISSION_LOCATION,RELIGION,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,GENDER,AGE,RUNTIME,CPT_CD,TICKET_ID_SEQ,PROCEDURE,INTIME,OUTTIME,LOS,LOS_ACTUAL
0,31,128652,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,CATHOLIC,WHITE,STATUS EPILEPTICUS,1,M,72,2108-08-24 23:27:00,99251,1,Consultations,2108-08-22 23:28:42,2108-08-30 21:59:20,7.9379,7 days 22:30:38
1,31,128652,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,CATHOLIC,WHITE,STATUS EPILEPTICUS,1,M,72,2108-08-24 23:27:00,94003,2,Pulmonary,2108-08-22 23:28:42,2108-08-30 21:59:20,7.9379,7 days 22:30:38
2,31,128652,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,CATHOLIC,WHITE,STATUS EPILEPTICUS,1,M,72,2108-08-24 23:27:00,94003,3,Pulmonary,2108-08-22 23:28:42,2108-08-30 21:59:20,7.9379,7 days 22:30:38
3,36,165660,ELECTIVE,PHYS REFERRAL/NORMAL DELI,NOT SPECIFIED,WHITE,VENTRAL HERNIA/SDA,0,M,73,2134-05-12 11:30:00,99222,1,Hospital inpatient services,2134-05-12 06:52:47,2134-05-16 15:14:20,4.3483,4 days 08:21:33
4,36,165660,ELECTIVE,PHYS REFERRAL/NORMAL DELI,NOT SPECIFIED,WHITE,VENTRAL HERNIA/SDA,0,M,73,2134-05-12 11:30:00,99291,2,Critical care services,2134-05-12 06:52:47,2134-05-16 15:14:20,4.3483,4 days 08:21:33


In [ ]:
wrong_los_df = merged_data[merged_data['RUNTIME'] < merged_data['OUTTIME']]
wrong_los_df['LOS_ACTUAL'] = wrong_los_df['RUNTIME'] - wrong_los_df['INTIME']

merged_data.loc[merged_data['RUNTIME'] < merged_data['OUTTIME'], 'LOS_ACTUAL'] = wrong_los_df['LOS_ACTUAL']

<ipython-input-247-bf3ccbfe1a43>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wrong_los_df['LOS_ACTUAL'] = wrong_los_df['RUNTIME'] - wrong_los_df['INTIME']


In [ ]:
merged_data.head()

,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,ADMISSION_LOCATION,RELIGION,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,GENDER,AGE,RUNTIME,CPT_CD,TICKET_ID_SEQ,PROCEDURE,INTIME,OUTTIME,LOS,LOS_ACTUAL
0,31,128652,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,CATHOLIC,WHITE,STATUS EPILEPTICUS,1,M,72,2108-08-24 23:27:00,99251,1,Consultations,2108-08-22 23:28:42,2108-08-30 21:59:20,7.9379,1 days 23:58:18
1,31,128652,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,CATHOLIC,WHITE,STATUS EPILEPTICUS,1,M,72,2108-08-24 23:27:00,94003,2,Pulmonary,2108-08-22 23:28:42,2108-08-30 21:59:20,7.9379,1 days 23:58:18
2,31,128652,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,CATHOLIC,WHITE,STATUS EPILEPTICUS,1,M,72,2108-08-24 23:27:00,94003,3,Pulmonary,2108-08-22 23:28:42,2108-08-30 21:59:20,7.9379,1 days 23:58:18
3,36,165660,ELECTIVE,PHYS REFERRAL/NORMAL DELI,NOT SPECIFIED,WHITE,VENTRAL HERNIA/SDA,0,M,73,2134-05-12 11:30:00,99222,1,Hospital inpatient services,2134-05-12 06:52:47,2134-05-16 15:14:20,4.3483,0 days 04:37:13
4,36,165660,ELECTIVE,PHYS REFERRAL/NORMAL DELI,NOT SPECIFIED,WHITE,VENTRAL HERNIA/SDA,0,M,73,2134-05-12 11:30:00,99291,2,Critical care services,2134-05-12 06:52:47,2134-05-16 15:14:20,4.3483,0 days 04:37:13


In [ ]:
merged_data['LOS'] = merged_data['LOS_ACTUAL'].dt.total_seconds()/86400
merged_data.drop(columns=['LOS_ACTUAL', 'INTIME', 'OUTTIME'], inplace=True)

<ipython-input-249-afd27c67a9e6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data['LOS'] = merged_data['LOS_ACTUAL'].dt.total_seconds()/86400
<ipython-input-249-afd27c67a9e6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data.drop(columns=['LOS_ACTUAL', 'INTIME', 'OUTTIME'], inplace=True)


In [ ]:
merged_data.head()

,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,ADMISSION_LOCATION,RELIGION,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,GENDER,AGE,RUNTIME,CPT_CD,TICKET_ID_SEQ,PROCEDURE,LOS
0,31,128652,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,CATHOLIC,WHITE,STATUS EPILEPTICUS,1,M,72,2108-08-24 23:27:00,99251,1,Consultations,1.998819
1,31,128652,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,CATHOLIC,WHITE,STATUS EPILEPTICUS,1,M,72,2108-08-24 23:27:00,94003,2,Pulmonary,1.998819
2,31,128652,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,CATHOLIC,WHITE,STATUS EPILEPTICUS,1,M,72,2108-08-24 23:27:00,94003,3,Pulmonary,1.998819
3,36,165660,ELECTIVE,PHYS REFERRAL/NORMAL DELI,NOT SPECIFIED,WHITE,VENTRAL HERNIA/SDA,0,M,73,2134-05-12 11:30:00,99222,1,Hospital inpatient services,0.192512
4,36,165660,ELECTIVE,PHYS REFERRAL/NORMAL DELI,NOT SPECIFIED,WHITE,VENTRAL HERNIA/SDA,0,M,73,2134-05-12 11:30:00,99291,2,Critical care services,0.192512


In [ ]:
# One HADM_ID can have multiple ICUSTAY_ID since a patient can be transferred into and out of an ICU multiple times
# So add up the length of stays corresponding to different ICUSTAY_ID and same HADM_ID
# merged_data = merged_data.groupby(by=list(merged_data.columns.drop('LOS'))).sum().reset_index().sort_values(by=['HADM_ID', 'TICKET_ID_SEQ'])
merged_data = merged_data.groupby(by=list(merged_data.columns.drop('LOS'))).sum().reset_index().sort_values(by=['HADM_ID'])
merged_data.head()

,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,ADMISSION_LOCATION,RELIGION,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,GENDER,AGE,RUNTIME,CPT_CD,TICKET_ID_SEQ,PROCEDURE,LOS
1718,533,100009,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,CATHOLIC,WHITE,CORONARY ARTERY DISEASE,0,M,61,2162-05-18 15:56:00,33508,1,Cardiovascular system,1.234363
1720,533,100009,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,CATHOLIC,WHITE,CORONARY ARTERY DISEASE,0,M,61,2162-05-18 15:56:00,33519,2,Cardiovascular system,1.234363
1721,533,100009,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,CATHOLIC,WHITE,CORONARY ARTERY DISEASE,0,M,61,2162-05-18 15:56:00,33519,9,Cardiovascular system,1.234363
1722,533,100009,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,CATHOLIC,WHITE,CORONARY ARTERY DISEASE,0,M,61,2162-05-18 15:56:00,33533,3,Cardiovascular system,1.234363
1723,533,100009,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,CATHOLIC,WHITE,CORONARY ARTERY DISEASE,0,M,61,2162-05-18 15:56:00,33533,10,Cardiovascular system,1.234363


In [ ]:
len(merged_data)

208714

In [ ]:
del wrong_los_df

In [ ]:
merged_data = merged_data.rename({'LOS': 'CURRENT_LOS'}, axis=1)

In [ ]:
icustays = icustays[['HADM_ID', 'LOS']]
icustays = icustays.groupby(by='HADM_ID').sum().reset_index()
merged_data = pd.merge(merged_data, icustays, how='inner', on=['HADM_ID'])

In [ ]:
merged_data.nunique()

SUBJECT_ID              12368
HADM_ID                 13700
ADMISSION_TYPE              3
ADMISSION_LOCATION          7
RELIGION                   19
ETHNICITY                  38
DIAGNOSIS                5175
HOSPITAL_EXPIRE_FLAG        2
GENDER                      2
AGE                        85
RUNTIME                 13674
CPT_CD                   2159
TICKET_ID_SEQ             284
PROCEDURE                  51
CURRENT_LOS              6465
LOS                     12845
dtype: int64

In [ ]:
merged_data.isnull().sum()

SUBJECT_ID              0
HADM_ID                 0
ADMISSION_TYPE          0
ADMISSION_LOCATION      0
RELIGION                0
ETHNICITY               0
DIAGNOSIS               0
HOSPITAL_EXPIRE_FLAG    0
GENDER                  0
AGE                     0
RUNTIME                 0
CPT_CD                  0
TICKET_ID_SEQ           0
PROCEDURE               0
CURRENT_LOS             0
LOS                     0
dtype: int64

In [ ]:
merged_data['LOS'].describe()

count    208714.000000
mean         14.413646
std          14.752262
min           0.041600
25%           4.500000
50%           9.810600
75%          19.429100
max         153.928000
Name: LOS, dtype: float64

In [ ]:
# merged_data.to_csv("/content/drive/MyDrive/mimic-iii-clinical-database-1.4/Merged_files/adm_pat_cpt_icu_v4.csv", index=False)

## INPUTEVENTS (Ignore) (Large)

In [ ]:
# inputevents_cv = read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/INPUTEVENTS_CV.csv.gz')
# len(inputevents_cv), inputevents_cv.nunique()

In [ ]:
# inputevents_cv.isnull().sum()

In [ ]:
# inputevents_cv

In [ ]:
# inputevents_mv = read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/INPUTEVENTS_MV.csv.gz')
# len(inputevents_mv), inputevents_mv.nunique()

In [ ]:
# inputevents_mv.isnull().sum()

In [ ]:
# inputevents_mv

## LABEVENTS (Ignore) (Large)

In [ ]:
# labevents = read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/LABEVENTS.csv.gz')
# len(labevents), labevents.nunique()

In [ ]:
# labevents.isnull().sum()

In [ ]:
# labevents

## MICROBIOLOGYEVENTS (Ignore)

In [ ]:
microbiologyevents = read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/MICROBIOLOGYEVENTS.csv.gz')
len(microbiologyevents), microbiologyevents.nunique()

(631726,
 ROW_ID                 631726
 SUBJECT_ID              39184
 HADM_ID                 48740
 CHARTDATE               37378
 CHARTTIME              292452
 SPEC_ITEMID                92
 SPEC_TYPE_DESC             87
 ORG_ITEMID                309
 ORG_NAME                  362
 ISOLATE_NUM                 6
 AB_ITEMID                  30
 AB_NAME                    30
 DILUTION_TEXT              47
 DILUTION_COMPARISON         3
 DILUTION_VALUE             18
 INTERPRETATION              4
 dtype: int64)

In [ ]:
microbiologyevents.isnull().sum()

ROW_ID                      0
SUBJECT_ID                  0
HADM_ID                     0
CHARTDATE                   0
CHARTTIME               41772
SPEC_ITEMID                79
SPEC_TYPE_DESC              0
ORG_ITEMID             304845
ORG_NAME               303710
ISOLATE_NUM            303710
AB_ITEMID              355892
AB_NAME                355892
DILUTION_TEXT          364376
DILUTION_COMPARISON    364548
DILUTION_VALUE         364548
INTERPRETATION         355892
dtype: int64

In [ ]:
microbiologyevents.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,SPEC_ITEMID,SPEC_TYPE_DESC,ORG_ITEMID,ORG_NAME,ISOLATE_NUM,AB_ITEMID,AB_NAME,DILUTION_TEXT,DILUTION_COMPARISON,DILUTION_VALUE,INTERPRETATION
0,744,96,170324,2156-04-13 00:00:00,2156-04-13 14:18:00,70021.0,BRONCHOALVEOLAR LAVAGE,80026.0,PSEUDOMONAS AERUGINOSA,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,745,96,170324,2156-04-20 00:00:00,2156-04-20 13:10:00,70062.0,SPUTUM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,746,96,170324,2156-04-20 00:00:00,2156-04-20 16:00:00,70012.0,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,747,96,170324,2156-04-20 00:00:00,NaN,70012.0,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,748,96,170324,2156-04-20 00:00:00,NaN,70079.0,URINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## NOTEEVENTS

In [ ]:
# merged_data = pd.read_csv("/content/drive/MyDrive/mimic-iii-clinical-database-1.4/Merged_files/adm_pat_cpt_icu_v3.csv")

In [ ]:
merged_data

,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,ADMISSION_LOCATION,RELIGION,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,GENDER,AGE,RUNTIME,CPT_CD,TICKET_ID_SEQ,PROCEDURE,CURRENT_LOS,LOS
0,533,100009,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,CATHOLIC,WHITE,CORONARY ARTERY DISEASE,0,M,61,2162-05-18 15:56:00,33508,1,Cardiovascular system,1.234363,2.4908
1,533,100009,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,CATHOLIC,WHITE,CORONARY ARTERY DISEASE,0,M,61,2162-05-18 15:56:00,33519,2,Cardiovascular system,1.234363,2.4908
2,533,100009,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,CATHOLIC,WHITE,CORONARY ARTERY DISEASE,0,M,61,2162-05-18 15:56:00,33519,9,Cardiovascular system,1.234363,2.4908
3,533,100009,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,CATHOLIC,WHITE,CORONARY ARTERY DISEASE,0,M,61,2162-05-18 15:56:00,33533,3,Cardiovascular system,1.234363,2.4908
4,533,100009,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,CATHOLIC,WHITE,CORONARY ARTERY DISEASE,0,M,61,2162-05-18 15:56:00,33533,10,Cardiovascular system,1.234363,2.4908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208709,41115,199979,ELECTIVE,PHYS REFERRAL/NORMAL DELI,PROTESTANT QUAKER,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,M,80,2182-02-07 03:30:00,33405,1,Cardiovascular system,0.662581,1.2868
208710,41115,199979,ELECTIVE,PHYS REFERRAL/NORMAL DELI,PROTESTANT QUAKER,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,M,80,2182-02-07 03:30:00,94002,7,Pulmonary,0.662581,1.2868
208711,41115,199979,ELECTIVE,PHYS REFERRAL/NORMAL DELI,PROTESTANT QUAKER,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,M,80,2182-02-07 03:30:00,99231,5,Hospital inpatient services,0.662581,1.2868
208712,41115,199979,ELECTIVE,PHYS REFERRAL/NORMAL DELI,PROTESTANT QUAKER,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,M,80,2182-02-07 03:30:00,33511,3,Cardiovascular system,0.662581,1.2868


In [ ]:
# Group by 'HADM_ID' and aggregate unique values for other columns
merged_data.drop(columns=['TICKET_ID_SEQ'], inplace=True)
merged_data = merged_data.groupby('HADM_ID').agg({
    'SUBJECT_ID': lambda x: ','.join(map(str, x.unique())),
    'ADMISSION_TYPE': lambda x: ','.join(map(str, x.unique())),
    'ADMISSION_LOCATION': lambda x: ','.join(map(str, x.unique())),
    'RELIGION': lambda x: ','.join(map(str, x.unique())),
    'ETHNICITY': lambda x: ','.join(map(str, x.unique())),
    'DIAGNOSIS': lambda x: ','.join(map(str, x.unique())),
    'HOSPITAL_EXPIRE_FLAG': lambda x: ','.join(map(str, x.unique())),
    'GENDER': lambda x: ','.join(map(str, x.unique())),
    'AGE': lambda x: ','.join(map(str, x.unique())),
    'RUNTIME': lambda x: ','.join(map(str, x.unique())),
    'CPT_CD': lambda x: ','.join(map(str, x)),
    'PROCEDURE': lambda x: ','.join(map(str, x)),
    'CURRENT_LOS': lambda x: ','.join(map(str, x.unique())),
    'LOS': lambda x: ','.join(map(str, x.unique()))
})

merged_data.reset_index(inplace=True)
merged_data.replace('M', 'Male', inplace=True)
merged_data.replace('F', 'Female', inplace=True)
merged_data = merged_data.applymap(lambda x: x.lower() if isinstance(x, str) else x)
merged_data.head()

,HADM_ID,SUBJECT_ID,ADMISSION_TYPE,ADMISSION_LOCATION,RELIGION,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,GENDER,AGE,RUNTIME,CPT_CD,PROCEDURE,CURRENT_LOS,LOS
0,100009,533,emergency,transfer from hosp/extram,catholic,white,coronary artery disease,0,male,61,2162-05-18 15:56:00,"33508,33519,33519,33533,33533,99291,99233,9923...","cardiovascular system,cardiovascular system,ca...",1.234363425925926,2.4908
1,100011,87977,emergency,clinic referral/premature,not specified,hispanic or latino,motor vehicle accident with unspecified injuries,0,male,21,2177-08-31 04:51:00,"99231,99231,99223,31624,31645,31500,28470,2752...","hospital inpatient services,hospital inpatient...",1.9990625,11.5029
2,100012,60039,emergency,transfer from hosp/extram,catholic,white,coronary artery disease,0,male,68,2177-03-14 11:48:00,"33518,94002,94002,33508,33508,33518,33533,3353...","cardiovascular system,pulmonary,pulmonary,card...",0.038622685185185184,3.5763
3,100016,68591,emergency,clinic referral/premature,protestant quaker,white,pneumonia,0,male,56,2188-05-26 13:06:00,"94003,31600,31646,99221,99222,99291,99291,9929...","pulmonary,respiratory system,respiratory syste...",1.999074074074074,6.1731
4,100018,58128,elective,phys referral/normal deli,protestant quaker,white,herniated disc/sda,0,male,55,2176-08-31 10:30:00,"63290,20931,94003,94003,94003,63043,69990,9592...","nervous system,musculoskeletal system,pulmonar...",1.7315162037037037,3.9222


In [ ]:
merged_data['RUNTIME'] = pd.to_datetime(merged_data['RUNTIME'])

In [ ]:
noteevents = read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/NOTEEVENTS.csv.gz')
len(noteevents), noteevents.nunique()

<ipython-input-101-27d2bf21da2b>:4: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(f)


(2083180,
 ROW_ID         2083180
 SUBJECT_ID       46146
 HADM_ID          58361
 CHARTDATE        38686
 CHARTTIME      1448491
 STORETIME      1224714
 CATEGORY            15
 DESCRIPTION       3848
 CGID              1912
 ISERROR              1
 TEXT           2023185
 dtype: int64)

In [ ]:
noteevents.isnull().sum()

ROW_ID               0
SUBJECT_ID           0
HADM_ID         231836
CHARTDATE            0
CHARTTIME       316566
STORETIME       836776
CATEGORY             0
DESCRIPTION          0
CGID            836776
ISERROR        2082294
TEXT                 0
dtype: int64

In [ ]:
noteevents.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [ ]:
noteevents.iloc[1, -1].split('\n')

['Admission Date:  [**2118-6-2**]       Discharge Date:  [**2118-6-14**]',
 '',
 'Date of Birth:                    Sex:  F',
 '',
 'Service:  MICU and then to [**Doctor Last Name **] Medicine',
 '',
 'HISTORY OF PRESENT ILLNESS:  This is an 81-year-old female',
 'with a history of emphysema (not on home O2), who presents',
 'with three days of shortness of breath thought by her primary',
 'care doctor to be a COPD flare.  Two days prior to admission,',
 'she was started on a prednisone taper and one day prior to',
 'admission she required oxygen at home in order to maintain',
 'oxygen saturation greater than 90%.  She has also been on',
 'levofloxacin and nebulizers, and was not getting better, and',
 'presented to the [**Hospital1 18**] Emergency Room.',
 '',
 'In the [**Hospital3 **] Emergency Room, her oxygen saturation was',
 '100% on CPAP.  She was not able to be weaned off of this',
 'despite nebulizer treatment and Solu-Medrol 125 mg IV x2.',
 '',
 'Review of systems is negativ

In [ ]:
noteevents = noteevents[~noteevents['HADM_ID'].isna()]      # Removing outpatient data
noteevents = noteevents[noteevents['ISERROR'] != 1.0]       # Removing erroneous notes

In [ ]:
noteevents.tail()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
2083175,2070657,31097,115637.0,2132-01-21,2132-01-21 03:27:00,2132-01-21 03:38:00,Nursing/other,Report,17581.0,NaN,NPN\n\n\n#1 Infant remains in RA with O2 sats...
2083176,2070658,31097,115637.0,2132-01-21,2132-01-21 09:50:00,2132-01-21 09:53:00,Nursing/other,Report,19211.0,NaN,"Neonatology\nDOL #5, CGA 36 weeks.\n\nCVR: Con..."
2083177,2070659,31097,115637.0,2132-01-21,2132-01-21 16:42:00,2132-01-21 16:44:00,Nursing/other,Report,20104.0,NaN,Family Meeting Note\nFamily meeting held with ...
2083178,2070660,31097,115637.0,2132-01-21,2132-01-21 18:05:00,2132-01-21 18:16:00,Nursing/other,Report,16023.0,NaN,NPN 1800\n\n\n#1 Resp: [**Known lastname 2243*...
2083179,2070661,31097,115637.0,2132-01-21,2132-01-21 18:05:00,2132-01-21 18:31:00,Nursing/other,Report,16023.0,NaN,NPN 1800\nNursing Addendum:\n[**Known lastname...


In [ ]:
noteevents = noteevents[['SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CATEGORY', 'DESCRIPTION', 'TEXT']]
noteevents['CHARTDATE'] = pd.to_datetime(noteevents['CHARTDATE'])
runtime_df = merged_data[['HADM_ID', 'RUNTIME']].drop_duplicates()
noteevents = pd.merge(noteevents, runtime_df, how='inner', on='HADM_ID')

In [ ]:
len(noteevents), noteevents.isnull().sum()

(628540,
 SUBJECT_ID     0
 HADM_ID        0
 CHARTDATE      0
 CATEGORY       0
 DESCRIPTION    0
 TEXT           0
 RUNTIME        0
 dtype: int64)

In [ ]:
noteevents = noteevents[noteevents['CHARTDATE'] < noteevents['RUNTIME']]
len(noteevents)

207563

In [ ]:
noteevents['CATEGORY'].value_counts()

CATEGORY
Radiology            63725
Nursing/other        55303
Nursing              30060
ECG                  23371
Physician            17743
Respiratory           6796
Echo                  6732
Nutrition             1451
General               1269
Social Work            391
Rehab Services         314
Discharge summary      269
Case Management         98
Consult                 30
Pharmacy                11
Name: count, dtype: int64

In [ ]:
noteevents.drop(columns=['CHARTDATE', 'RUNTIME'], inplace=True)

merged_data['SUBJECT_ID'] = merged_data['SUBJECT_ID'].astype(int)
merged_data = pd.merge(merged_data, noteevents, how='inner', on=['SUBJECT_ID', 'HADM_ID'])
merged_data

<ipython-input-275-b3808e5c5164>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noteevents.drop(columns=['CHARTDATE', 'RUNTIME'], inplace=True)


,HADM_ID,SUBJECT_ID,ADMISSION_TYPE,ADMISSION_LOCATION,RELIGION,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,GENDER,AGE,RUNTIME,CPT_CD,PROCEDURE,CURRENT_LOS,LOS,CATEGORY,DESCRIPTION,TEXT
0,100009,533,emergency,transfer from hosp/extram,catholic,white,coronary artery disease,0,male,61,2162-05-18 15:56:00,"33508,33519,33519,33533,33533,99291,99233,9923...","cardiovascular system,cardiovascular system,ca...",1.234363425925926,2.4908,Echo,Report,PATIENT/TEST INFORMATION:\nIndication: Abnorma...
1,100009,533,emergency,transfer from hosp/extram,catholic,white,coronary artery disease,0,male,61,2162-05-18 15:56:00,"33508,33519,33519,33533,33533,99291,99233,9923...","cardiovascular system,cardiovascular system,ca...",1.234363425925926,2.4908,ECG,Report,Sinus bradycardia. Left atrial abnormality. ...
2,100009,533,emergency,transfer from hosp/extram,catholic,white,coronary artery disease,0,male,61,2162-05-18 15:56:00,"33508,33519,33519,33533,33533,99291,99233,9923...","cardiovascular system,cardiovascular system,ca...",1.234363425925926,2.4908,Radiology,CHEST (PA & LAT),[**2162-5-16**] 7:23 PM\n CHEST (PA & LAT) ...
3,100009,533,emergency,transfer from hosp/extram,catholic,white,coronary artery disease,0,male,61,2162-05-18 15:56:00,"33508,33519,33519,33533,33533,99291,99233,9923...","cardiovascular system,cardiovascular system,ca...",1.234363425925926,2.4908,Radiology,VEN DUP EXTEXT BIL (MAP/DVT),[**2162-5-17**] 8:12 AM\n [**Last Name (un) 12...
4,100009,533,emergency,transfer from hosp/extram,catholic,white,coronary artery disease,0,male,61,2162-05-18 15:56:00,"33508,33519,33519,33533,33533,99291,99233,9923...","cardiovascular system,cardiovascular system,ca...",1.234363425925926,2.4908,Radiology,CHEST PORT. LINE PLACEMENT,[**2162-5-17**] 6:41 PM\n CHEST PORT. LINE PLA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207558,199998,27200,emergency,transfer from hosp/extram,catholic,white,coronary artery disease,0,male,70,2119-02-20 16:26:00,94002,pulmonary,0.2648263888888889,1.0623,Radiology,CHEST PORT. LINE PLACEMENT,[**2119-2-20**] 4:09 PM\n CHEST PORT. LINE PLA...
207559,199998,27200,emergency,transfer from hosp/extram,catholic,white,coronary artery disease,0,male,70,2119-02-20 16:26:00,94002,pulmonary,0.2648263888888889,1.0623,Radiology,CT CHEST W/O CONTRAST,[**2119-2-18**] 5:59 PM\n CT CHEST W/O CONTRAS...
207560,199998,27200,emergency,transfer from hosp/extram,catholic,white,coronary artery disease,0,male,70,2119-02-20 16:26:00,94002,pulmonary,0.2648263888888889,1.0623,Radiology,CHEST (PRE-OP PA & LAT),[**2119-2-18**] 5:49 PM\n CHEST (PRE-OP PA & L...
207561,199998,27200,emergency,transfer from hosp/extram,catholic,white,coronary artery disease,0,male,70,2119-02-20 16:26:00,94002,pulmonary,0.2648263888888889,1.0623,Nursing/other,Report,Resp Care\n\nPt received from OR and placed on...


In [ ]:
# merged_data.to_csv("/content/drive/MyDrive/mimic-iii-clinical-database-1.4/Merged_files/adm_pat_cpt_icu_note_v4.csv", index=False)

## OUTPUTEVENTS (Ignore)

In [ ]:
outputevents = read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/OUTPUTEVENTS.csv.gz')
len(outputevents), outputevents.nunique()

(4349218,
 ROW_ID        4349218
 SUBJECT_ID      41857
 HADM_ID         52008
 ICUSTAY_ID      55076
 CHARTTIME     1172101
 ITEMID           1155
 VALUE            2114
 VALUEUOM            2
 STORETIME     3201069
 CGID             1551
 STOPPED             0
 NEWBOTTLE           0
 ISERROR             0
 dtype: int64)

In [ ]:
outputevents.isnull().sum()

ROW_ID              0
SUBJECT_ID          0
HADM_ID          4839
ICUSTAY_ID       7190
CHARTTIME           0
ITEMID              0
VALUE           88549
VALUEUOM        36251
STORETIME           0
CGID                0
STOPPED       4349218
NEWBOTTLE     4349218
ISERROR       4349218
dtype: int64

In [ ]:
outputevents.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR
0,344,21219,177991.0,225765.0,2142-09-08 10:00:00,40055,200.0,ml,2142-09-08 12:08:00,17269,NaN,NaN,NaN
1,345,21219,177991.0,225765.0,2142-09-08 12:00:00,40055,200.0,ml,2142-09-08 12:08:00,17269,NaN,NaN,NaN
2,346,21219,177991.0,225765.0,2142-09-08 13:00:00,40055,120.0,ml,2142-09-08 13:39:00,17269,NaN,NaN,NaN
3,347,21219,177991.0,225765.0,2142-09-08 14:00:00,40055,100.0,ml,2142-09-08 16:17:00,17269,NaN,NaN,NaN
4,348,21219,177991.0,225765.0,2142-09-08 16:00:00,40055,200.0,ml,2142-09-08 16:17:00,17269,NaN,NaN,NaN


## PRESCRIPTIONS (Ignore)

In [ ]:
prescriptions = read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/PRESCRIPTIONS.csv.gz')
len(prescriptions), prescriptions.nunique()

<ipython-input-2-27d2bf21da2b>:4: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(f)


(4156450,
 ROW_ID               4156450
 SUBJECT_ID             39363
 HADM_ID                50216
 ICUSTAY_ID             52151
 STARTDATE              38497
 ENDDATE                38500
 DRUG_TYPE                  3
 DRUG                    4525
 DRUG_NAME_POE           4036
 DRUG_NAME_GENERIC       2863
 FORMULARY_DRUG_CD       3267
 GSN                     4685
 NDC                     4204
 PROD_STRENGTH           4000
 DOSE_VAL_RX             2605
 DOSE_UNIT_RX             104
 FORM_VAL_DISP           3073
 FORM_UNIT_DISP            84
 ROUTE                     78
 dtype: int64)

In [ ]:
prescriptions.isnull().sum()

ROW_ID                     0
SUBJECT_ID                 0
HADM_ID                    0
ICUSTAY_ID           1447708
STARTDATE               3182
ENDDATE                 5421
DRUG_TYPE                  0
DRUG                       0
DRUG_NAME_POE        1664234
DRUG_NAME_GENERIC    1662989
FORMULARY_DRUG_CD       1933
GSN                   507164
NDC                     4463
PROD_STRENGTH           1362
DOSE_VAL_RX             1350
DOSE_UNIT_RX            1342
FORM_VAL_DISP           1355
FORM_UNIT_DISP          1409
ROUTE                   1156
dtype: int64

In [ ]:
prescriptions.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,469061711.0,1mg Capsule,2,mg,2,CAP,PO
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,56017275.0,5mg Tablet,5,mg,1,TAB,PO
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,338055002.0,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.0,HEPARIN BASE,250,ml,250,ml,IV
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,54829725.0,20mg Tablet,20,mg,1,TAB,PO


## PROCEDUREEVENTS_MV (Ignore)

In [ ]:
procedureevents_mv = read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/PROCEDUREEVENTS_MV.csv.gz')
len(procedureevents_mv), procedureevents_mv.nunique()

(258066,
 ROW_ID                        258066
 SUBJECT_ID                     17692
 HADM_ID                        21894
 ICUSTAY_ID                     23401
 STARTTIME                     218670
 ENDTIME                       222925
 ITEMID                           116
 VALUE                          13510
 VALUEUOM                           3
 LOCATION                          97
 LOCATIONCATEGORY                   8
 STORETIME                     206719
 CGID                             991
 ORDERID                       258066
 LINKORDERID                   257837
 ORDERCATEGORYNAME                 12
 SECONDARYORDERCATEGORYNAME         0
 ORDERCATEGORYDESCRIPTION           2
 ISOPENBAG                          2
 CONTINUEINNEXTDEPT                 2
 CANCELREASON                       3
 STATUSDESCRIPTION                  4
 COMMENTS_EDITEDBY                  6
 COMMENTS_CANCELEDBY               17
 COMMENTS_DATE                   6929
 dtype: int64)

In [ ]:
procedureevents_mv.isnull().sum()

ROW_ID                             0
SUBJECT_ID                         0
HADM_ID                            0
ICUSTAY_ID                        88
STARTTIME                          0
ENDTIME                            0
ITEMID                             0
VALUE                              0
VALUEUOM                      144246
LOCATION                      205454
LOCATIONCATEGORY              205454
STORETIME                          0
CGID                               0
ORDERID                            0
LINKORDERID                        0
ORDERCATEGORYNAME                  0
SECONDARYORDERCATEGORYNAME    258066
ORDERCATEGORYDESCRIPTION           0
ISOPENBAG                          0
CONTINUEINNEXTDEPT                 0
CANCELREASON                       0
STATUSDESCRIPTION                  0
COMMENTS_EDITEDBY             255973
COMMENTS_CANCELEDBY           252377
COMMENTS_DATE                 250284
dtype: int64

In [ ]:
procedureevents_mv.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,VALUE,VALUEUOM,LOCATION,...,ORDERCATEGORYNAME,SECONDARYORDERCATEGORYNAME,ORDERCATEGORYDESCRIPTION,ISOPENBAG,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE
0,379,29070,115071,232563.0,2145-03-12 23:04:00,2145-03-12 23:05:00,225401,1.0,NaN,NaN,...,Procedures,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN
1,380,29070,115071,232563.0,2145-03-12 23:04:00,2145-03-12 23:05:00,225454,1.0,NaN,NaN,...,Procedures,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN
2,381,29070,115071,232563.0,2145-03-12 23:05:00,2145-03-18 20:01:00,225792,8456.0,hour,NaN,...,Ventilation,NaN,Task,1,0,0,FinishedRunning,NaN,NaN,NaN
3,382,29070,115071,232563.0,2145-03-12 23:36:00,2145-03-12 23:37:00,225402,1.0,NaN,NaN,...,Procedures,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN
4,383,29070,115071,232563.0,2145-03-13 01:27:00,2145-03-16 16:00:00,224560,5193.0,min,Right IJ,...,Invasive Lines,NaN,Task,1,0,0,FinishedRunning,NaN,NaN,NaN


## PROCEDURES_ICD (Ignore)

In [ ]:
procedures_icd = read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/PROCEDURES_ICD.csv.gz')
len(procedures_icd), procedures_icd.nunique()

(240095,
 ROW_ID        240095
 SUBJECT_ID     42214
 HADM_ID        52243
 SEQ_NUM           40
 ICD9_CODE       2009
 dtype: int64)

In [ ]:
procedures_icd.isnull().sum()

ROW_ID        0
SUBJECT_ID    0
HADM_ID       0
SEQ_NUM       0
ICD9_CODE     0
dtype: int64

In [ ]:
procedures_icd.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,944,62641,154460,3,3404
1,945,2592,130856,1,9671
2,946,2592,130856,2,3893
3,947,55357,119355,1,9672
4,948,55357,119355,2,331


## SERVICES

In [ ]:
services = read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/SERVICES.csv.gz')
len(services), services.nunique()

(73343,
 ROW_ID          73343
 SUBJECT_ID      46496
 HADM_ID         58926
 TRANSFERTIME    73336
 PREV_SERVICE       18
 CURR_SERVICE       20
 dtype: int64)

In [ ]:
services.isnull().sum()

ROW_ID              0
SUBJECT_ID          0
HADM_ID             0
TRANSFERTIME        0
PREV_SERVICE    58675
CURR_SERVICE        0
dtype: int64

In [ ]:
services.head()

,ROW_ID,SUBJECT_ID,HADM_ID,TRANSFERTIME,PREV_SERVICE,CURR_SERVICE
0,758,471,135879,2122-07-22 14:07:27,TSURG,MED
1,759,471,135879,2122-07-26 18:31:49,MED,TSURG
2,760,472,173064,2172-09-28 19:22:15,NaN,CMED
3,761,473,129194,2201-01-09 20:16:45,NaN,NB
4,762,474,194246,2181-03-23 08:24:41,NaN,NB


## TRANSFERS

In [ ]:
transfers = read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/TRANSFERS.csv.gz')
len(transfers), transfers.nunique()

(261897,
 ROW_ID           261897
 SUBJECT_ID        46520
 HADM_ID           58976
 ICUSTAY_ID        61532
 DBSOURCE              3
 EVENTTYPE             3
 PREV_CAREUNIT         7
 CURR_CAREUNIT         7
 PREV_WARDID          53
 CURR_WARDID          53
 INTIME           261775
 OUTTIME          202830
 LOS               29993
 dtype: int64)

In [ ]:
transfers.isnull().sum()

ROW_ID                0
SUBJECT_ID            0
HADM_ID               0
ICUSTAY_ID       174176
DBSOURCE            241
EVENTTYPE            24
PREV_CAREUNIT    165888
CURR_CAREUNIT    165881
PREV_WARDID       58933
CURR_WARDID       58943
INTIME               24
OUTTIME           58976
LOS               58976
dtype: int64

In [ ]:
transfers.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,EVENTTYPE,PREV_CAREUNIT,CURR_CAREUNIT,PREV_WARDID,CURR_WARDID,INTIME,OUTTIME,LOS
0,657,111,192123,254245.0,carevue,transfer,CCU,MICU,7.0,23.0,2142-04-29 15:27:11,2142-05-04 20:38:33,125.19
1,658,111,192123,NaN,carevue,transfer,MICU,NaN,23.0,45.0,2142-05-04 20:38:33,2142-05-05 11:46:32,15.13
2,659,111,192123,NaN,carevue,discharge,NaN,NaN,45.0,NaN,2142-05-05 11:46:32,NaN,NaN
3,660,111,155897,249202.0,metavision,admit,NaN,MICU,NaN,52.0,2144-07-01 04:13:59,2144-07-01 05:19:39,1.09
4,661,111,155897,NaN,metavision,transfer,MICU,NaN,52.0,32.0,2144-07-01 05:19:39,2144-07-01 06:28:29,1.15


# Old

## Merging CPTEVENTS and ADMISSIONS

In [ ]:
cptevents_admissions = pd.merge(cptevents, admissions[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ADMISSION_TYPE', 'DISCHARGE_LOCATION', 'DIAGNOSIS']],
                                how='inner', on=['SUBJECT_ID', 'HADM_ID'])


In [ ]:
cptevents_admissions

,ROW_ID,SUBJECT_ID,HADM_ID,COSTCENTER,CHARTDATE,CPT_CD,CPT_NUMBER,CPT_SUFFIX,TICKET_ID_SEQ,SECTIONHEADER,SUBSECTIONHEADER,DESCRIPTION,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,DISCHARGE_LOCATION,DIAGNOSIS
0,317,11743,129545,ICU,NaN,99232,99232.0,NaN,6.0,Evaluation and management,Hospital inpatient services,NaN,2147-02-24 00:13:00,2147-03-17 12:51:00,NaN,EMERGENCY,REHAB/DISTINCT PART HOSP,HEMATEMESIS
1,318,11743,129545,ICU,NaN,99232,99232.0,NaN,7.0,Evaluation and management,Hospital inpatient services,NaN,2147-02-24 00:13:00,2147-03-17 12:51:00,NaN,EMERGENCY,REHAB/DISTINCT PART HOSP,HEMATEMESIS
2,319,11743,129545,ICU,NaN,99232,99232.0,NaN,8.0,Evaluation and management,Hospital inpatient services,NaN,2147-02-24 00:13:00,2147-03-17 12:51:00,NaN,EMERGENCY,REHAB/DISTINCT PART HOSP,HEMATEMESIS
3,320,11743,129545,ICU,NaN,99232,99232.0,NaN,9.0,Evaluation and management,Hospital inpatient services,NaN,2147-02-24 00:13:00,2147-03-17 12:51:00,NaN,EMERGENCY,REHAB/DISTINCT PART HOSP,HEMATEMESIS
4,308,11743,129545,ICU,NaN,99223,99223.0,NaN,1.0,Evaluation and management,Hospital inpatient services,NaN,2147-02-24 00:13:00,2147-03-17 12:51:00,NaN,EMERGENCY,REHAB/DISTINCT PART HOSP,HEMATEMESIS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573141,572152,86585,123444,Resp,2137-12-13 00:00:00,94003,94003.0,NaN,NaN,Medicine,Pulmonary,VENT MGMT;SUBSQ DAYS(INVASIVE),2137-12-09 21:16:00,2137-12-20 16:45:00,NaN,EMERGENCY,HOME HEALTH CARE,RESPIRATORY FAILURE;CONGESTIVE HEART FAILURE
573142,572153,86585,123444,Resp,2137-12-11 00:00:00,94003,94003.0,NaN,NaN,Medicine,Pulmonary,VENT MGMT;SUBSQ DAYS(INVASIVE),2137-12-09 21:16:00,2137-12-20 16:45:00,NaN,EMERGENCY,HOME HEALTH CARE,RESPIRATORY FAILURE;CONGESTIVE HEART FAILURE
573143,572154,86585,123444,Resp,2137-12-10 00:00:00,94002,94002.0,NaN,NaN,Medicine,Pulmonary,"VENT MGMT, 1ST DAY (INVASIVE)",2137-12-09 21:16:00,2137-12-20 16:45:00,NaN,EMERGENCY,HOME HEALTH CARE,RESPIRATORY FAILURE;CONGESTIVE HEART FAILURE
573144,572155,86585,123444,Resp,2137-12-10 00:00:00,94003,94003.0,NaN,NaN,Medicine,Pulmonary,VENT MGMT;SUBSQ DAYS(INVASIVE),2137-12-09 21:16:00,2137-12-20 16:45:00,NaN,EMERGENCY,HOME HEALTH CARE,RESPIRATORY FAILURE;CONGESTIVE HEART FAILURE


In [ ]:
patients=read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/PATIENTS.csv.gz')
patients

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
4,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...
46515,31840,44089,M,2026-05-25 00:00:00,NaN,NaN,NaN,0
46516,31841,44115,F,2124-07-27 00:00:00,NaN,NaN,NaN,0
46517,31842,44123,F,2049-11-26 00:00:00,2135-01-12 00:00:00,2135-01-12 00:00:00,NaN,1
46518,31843,44126,F,2076-07-25 00:00:00,NaN,NaN,NaN,0


## Merging cptevents + admissions + patients

In [ ]:
cptevents_admissions_patients = pd.merge(cptevents_admissions, patients[['SUBJECT_ID', 'GENDER', 'DOB', 'DOD']],
                                         how='inner', on=['SUBJECT_ID'])

In [ ]:
cptevents_admissions_patients

,ROW_ID,SUBJECT_ID,HADM_ID,COSTCENTER,CHARTDATE,CPT_CD,CPT_NUMBER,CPT_SUFFIX,TICKET_ID_SEQ,SECTIONHEADER,...,DESCRIPTION,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,DISCHARGE_LOCATION,DIAGNOSIS,GENDER,DOB,DOD
0,317,11743,129545,ICU,NaN,99232,99232.0,NaN,6.0,Evaluation and management,...,NaN,2147-02-24 00:13:00,2147-03-17 12:51:00,NaN,EMERGENCY,REHAB/DISTINCT PART HOSP,HEMATEMESIS,M,2066-05-22 00:00:00,2150-06-11 00:00:00
1,318,11743,129545,ICU,NaN,99232,99232.0,NaN,7.0,Evaluation and management,...,NaN,2147-02-24 00:13:00,2147-03-17 12:51:00,NaN,EMERGENCY,REHAB/DISTINCT PART HOSP,HEMATEMESIS,M,2066-05-22 00:00:00,2150-06-11 00:00:00
2,319,11743,129545,ICU,NaN,99232,99232.0,NaN,8.0,Evaluation and management,...,NaN,2147-02-24 00:13:00,2147-03-17 12:51:00,NaN,EMERGENCY,REHAB/DISTINCT PART HOSP,HEMATEMESIS,M,2066-05-22 00:00:00,2150-06-11 00:00:00
3,320,11743,129545,ICU,NaN,99232,99232.0,NaN,9.0,Evaluation and management,...,NaN,2147-02-24 00:13:00,2147-03-17 12:51:00,NaN,EMERGENCY,REHAB/DISTINCT PART HOSP,HEMATEMESIS,M,2066-05-22 00:00:00,2150-06-11 00:00:00
4,308,11743,129545,ICU,NaN,99223,99223.0,NaN,1.0,Evaluation and management,...,NaN,2147-02-24 00:13:00,2147-03-17 12:51:00,NaN,EMERGENCY,REHAB/DISTINCT PART HOSP,HEMATEMESIS,M,2066-05-22 00:00:00,2150-06-11 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573141,572151,86585,123444,Resp,2137-12-14 00:00:00,94003,94003.0,NaN,NaN,Medicine,...,VENT MGMT;SUBSQ DAYS(INVASIVE),2137-12-09 21:16:00,2137-12-20 16:45:00,NaN,EMERGENCY,HOME HEALTH CARE,RESPIRATORY FAILURE;CONGESTIVE HEART FAILURE,M,2056-01-24 00:00:00,2139-10-18 00:00:00
573142,572152,86585,123444,Resp,2137-12-13 00:00:00,94003,94003.0,NaN,NaN,Medicine,...,VENT MGMT;SUBSQ DAYS(INVASIVE),2137-12-09 21:16:00,2137-12-20 16:45:00,NaN,EMERGENCY,HOME HEALTH CARE,RESPIRATORY FAILURE;CONGESTIVE HEART FAILURE,M,2056-01-24 00:00:00,2139-10-18 00:00:00
573143,572153,86585,123444,Resp,2137-12-11 00:00:00,94003,94003.0,NaN,NaN,Medicine,...,VENT MGMT;SUBSQ DAYS(INVASIVE),2137-12-09 21:16:00,2137-12-20 16:45:00,NaN,EMERGENCY,HOME HEALTH CARE,RESPIRATORY FAILURE;CONGESTIVE HEART FAILURE,M,2056-01-24 00:00:00,2139-10-18 00:00:00
573144,572154,86585,123444,Resp,2137-12-10 00:00:00,94002,94002.0,NaN,NaN,Medicine,...,"VENT MGMT, 1ST DAY (INVASIVE)",2137-12-09 21:16:00,2137-12-20 16:45:00,NaN,EMERGENCY,HOME HEALTH CARE,RESPIRATORY FAILURE;CONGESTIVE HEART FAILURE,M,2056-01-24 00:00:00,2139-10-18 00:00:00


In [ ]:
diagnoses_icd=read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/DIAGNOSES_ICD.csv.gz')
diagnoses_icd

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254
...,...,...,...,...,...
651042,639798,97503,188195,2.0,20280
651043,639799,97503,188195,3.0,V5869
651044,639800,97503,188195,4.0,V1279
651045,639801,97503,188195,5.0,5275


##Patients + admission

In [ ]:
patients_admissions = pd.merge(patients, admissions[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'ADMISSION_TYPE', 'DISCHARGE_LOCATION', 'DIAGNOSIS']],
                               how='inner', on=['SUBJECT_ID'])
patients_admissions

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,DISCHARGE_LOCATION,DIAGNOSIS
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0,116935,2149-12-17 20:41:00,2149-12-31 14:55:00,EMERGENCY,SNF,UNSTABLE ANGINA;ASTHMA;BRONCHITIS
1,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0,149546,2155-02-03 20:16:00,2155-02-14 11:15:00,EMERGENCY,REHAB/DISTINCT PART HOSP,GI BLEED/ CHEST PAIN
2,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0,158975,2156-04-27 15:33:00,2156-05-14 15:30:00,EMERGENCY,SNF,GI BLEEDING\COLONOSCOPY
3,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1,124271,2188-11-12 09:22:00,2188-11-22 12:00:00,EMERGENCY,DEAD/EXPIRED,PNEUMONIA;R/O TB
4,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0,117937,2110-07-27 06:46:00,2110-07-29 15:23:00,EMERGENCY,HOME,INTRACRANIAL HEAD BLEED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,31840,44089,M,2026-05-25 00:00:00,NaN,NaN,NaN,0,165748,2111-09-30 12:04:00,2111-10-03 16:04:00,EMERGENCY,HOME HEALTH CARE,CHEST PAIN;ANGINA\LEFT HEART CATHETERIZATION
58972,31841,44115,F,2124-07-27 00:00:00,NaN,NaN,NaN,0,163623,2161-07-15 12:00:00,2161-07-19 12:30:00,ELECTIVE,HOME,CHIARI MALFORMATION BILATERAL/SDA
58973,31842,44123,F,2049-11-26 00:00:00,2135-01-12 00:00:00,2135-01-12 00:00:00,NaN,1,116395,2135-01-06 07:15:00,2135-01-12 02:50:00,ELECTIVE,DEAD/EXPIRED,ABDOMINAL ANEURYSM; NO RUPTURE/SDA
58974,31843,44126,F,2076-07-25 00:00:00,NaN,NaN,NaN,0,183530,2129-01-03 07:15:00,2129-01-11 12:29:00,ELECTIVE,REHAB/DISTINCT PART HOSP,SCOLIOSIS/SDA


##patients+admissions+diagnoses


In [ ]:
patients_admissions_diagnoses = pd.merge(patients_admissions, diagnoses_icd[['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE', 'SEQ_NUM']],
                                         how='inner', on=['SUBJECT_ID', 'HADM_ID'])
patients_admissions_diagnoses

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,DISCHARGE_LOCATION,DIAGNOSIS,ICD9_CODE,SEQ_NUM
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0,116935,2149-12-17 20:41:00,2149-12-31 14:55:00,EMERGENCY,SNF,UNSTABLE ANGINA;ASTHMA;BRONCHITIS,49322,1.0
1,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0,116935,2149-12-17 20:41:00,2149-12-31 14:55:00,EMERGENCY,SNF,UNSTABLE ANGINA;ASTHMA;BRONCHITIS,51882,2.0
2,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0,116935,2149-12-17 20:41:00,2149-12-31 14:55:00,EMERGENCY,SNF,UNSTABLE ANGINA;ASTHMA;BRONCHITIS,41071,3.0
3,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0,116935,2149-12-17 20:41:00,2149-12-31 14:55:00,EMERGENCY,SNF,UNSTABLE ANGINA;ASTHMA;BRONCHITIS,42841,4.0
4,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0,116935,2149-12-17 20:41:00,2149-12-31 14:55:00,EMERGENCY,SNF,UNSTABLE ANGINA;ASTHMA;BRONCHITIS,99812,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651042,31844,44128,M,2098-07-25 00:00:00,NaN,NaN,NaN,0,141304,2149-06-08 15:21:00,2149-06-23 18:44:00,EMERGENCY,REHAB/DISTINCT PART HOSP,TRAUMA,27651,19.0
651043,31844,44128,M,2098-07-25 00:00:00,NaN,NaN,NaN,0,141304,2149-06-08 15:21:00,2149-06-23 18:44:00,EMERGENCY,REHAB/DISTINCT PART HOSP,TRAUMA,27651,20.0
651044,31844,44128,M,2098-07-25 00:00:00,NaN,NaN,NaN,0,141304,2149-06-08 15:21:00,2149-06-23 18:44:00,EMERGENCY,REHAB/DISTINCT PART HOSP,TRAUMA,3051,21.0
651045,31844,44128,M,2098-07-25 00:00:00,NaN,NaN,NaN,0,141304,2149-06-08 15:21:00,2149-06-23 18:44:00,EMERGENCY,REHAB/DISTINCT PART HOSP,TRAUMA,56400,22.0


In [ ]:
patients_admissions_diagnoses.to_csv('patients_admissions_diagnoses.csv')

In [ ]:
prescriptions=read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/PRESCRIPTIONS.csv.gz')
prescriptions

<ipython-input-2-27d2bf21da2b>:4: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(f)


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,4.690617e+08,1mg Capsule,2,mg,2,CAP,PO
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,5.601728e+07,5mg Tablet,5,mg,1,TAB,PO
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,3.380550e+08,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.000000e+00,HEPARIN BASE,250,ml,250,ml,IV
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,5.482972e+07,20mg Tablet,20,mg,1,TAB,PO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4156445,3996662,98887,121032,238144.0,2144-09-06 00:00:00,2144-09-06 00:00:00,MAIN,PredniSONE,PredniSONE,PredniSONE,PRED20,006751,5.400182e+07,20 mg Tablet,40,mg,2,TAB,PO/NG
4156446,3996070,98887,121032,238144.0,2144-09-06 00:00:00,2144-09-06 00:00:00,MAIN,Ipratropium Bromide Neb,Ipratropium Bromide Neb,Ipratropium Bromide Neb,IPRA2H,021700,4.879801e+08,2.5mL Vial,1,NEB,1,VIAL,IH
4156447,3996063,98887,121032,238144.0,2144-09-06 00:00:00,2144-09-06 00:00:00,MAIN,HYDROmorphone (Dilaudid),HYDROmorphone (Dilaudid),HYDROmorphone,HYDR20/100NS,048078,6.155302e+10,20 mg / 100 mL Premix Bag,0.12,mg,0.01,BAG,IVPCA
4156448,3996062,98887,121032,238144.0,2144-09-06 00:00:00,2144-09-06 00:00:00,MAIN,Docusate Sodium,Docusate Sodium,Docusate Sodium,DOCU100,003009,9.042245e+08,100mg Capsule,100,mg,1,CAP,PO


In [ ]:
icustays=pd.read_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/ICUSTAYS.csv')
icustays

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202
...,...,...,...,...,...,...,...,...,...,...,...,...
61527,59806,94944,143774,201233,metavision,CSRU,CSRU,15,15,2104-04-15 10:18:16,2104-04-17 14:51:00,2.1894
61528,59807,94950,123750,283653,metavision,CCU,CCU,7,7,2155-12-08 05:33:16,2155-12-10 17:24:58,2.4942
61529,59808,94953,196881,241585,metavision,SICU,SICU,57,57,2160-03-03 16:09:11,2160-03-04 14:22:33,0.9259
61530,59809,94954,118475,202802,metavision,CSRU,CSRU,15,15,2183-03-25 09:53:10,2183-03-27 17:55:03,2.3346


In [ ]:
diagnoses_icd=read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/DRGCODES.csv.gz')
diagnoses_icd

,ROW_ID,SUBJECT_ID,HADM_ID,DRG_TYPE,DRG_CODE,DESCRIPTION,DRG_SEVERITY,DRG_MORTALITY
0,342,2491,144486,HCFA,28,"TRAUMATIC STUPOR & COMA, COMA <1 HR AGE >17 WI...",NaN,NaN
1,343,24958,162910,HCFA,110,MAJOR CARDIOVASCULAR PROCEDURES WITH COMPLICAT...,NaN,NaN
2,344,18325,153751,HCFA,390,NEONATE WITH OTHER SIGNIFICANT PROBLEMS,NaN,NaN
3,345,17887,182692,HCFA,14,SPECIFIC CEREBROVASCULAR DISORDERS EXCEPT TRAN...,NaN,NaN
4,346,11113,157980,HCFA,390,NEONATE WITH OTHER SIGNIFICANT PROBLEMS,NaN,NaN
...,...,...,...,...,...,...,...,...
125552,123452,71582,101422,MS,221,CARDIAC VALVE & OTH MAJ CARDIOTHORACIC PROC W/...,NaN,NaN
125553,123453,46449,110075,APR,1653,Coronary Bypass w/ Cardiac Cath Or Percutaneou...,3.0,2.0
125554,123454,46449,110075,APR,1653,Coronary Bypass w/ Cardiac Cath Or Percutaneou...,3.0,2.0
125555,123455,46449,110075,MS,234,CORONARY BYPASS W CARDIAC CATH W/O MCC,NaN,NaN


In [ ]:
icustays=pd.read_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/ICUSTAYS.csv')
icustays

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202
...,...,...,...,...,...,...,...,...,...,...,...,...
61527,59806,94944,143774,201233,metavision,CSRU,CSRU,15,15,2104-04-15 10:18:16,2104-04-17 14:51:00,2.1894
61528,59807,94950,123750,283653,metavision,CCU,CCU,7,7,2155-12-08 05:33:16,2155-12-10 17:24:58,2.4942
61529,59808,94953,196881,241585,metavision,SICU,SICU,57,57,2160-03-03 16:09:11,2160-03-04 14:22:33,0.9259
61530,59809,94954,118475,202802,metavision,CSRU,CSRU,15,15,2183-03-25 09:53:10,2183-03-27 17:55:03,2.3346


In [ ]:
labevents=read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/LABEVENTS.csv.gz')
labevents

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
0,281,3,NaN,50820,2101-10-12 16:07:00,7.39,7.39,units,NaN
1,282,3,NaN,50800,2101-10-12 18:17:00,ART,NaN,NaN,NaN
2,283,3,NaN,50802,2101-10-12 18:17:00,-1,-1.00,mEq/L,NaN
3,284,3,NaN,50804,2101-10-12 18:17:00,22,22.00,mEq/L,NaN
4,285,3,NaN,50808,2101-10-12 18:17:00,0.93,0.93,mmol/L,abnormal
...,...,...,...,...,...,...,...,...,...
27854050,27428435,96443,103219.0,50882,2109-12-30 01:40:00,26,26.00,mEq/L,NaN
27854051,27428436,96443,103219.0,50885,2109-12-30 01:40:00,2.1,2.10,mg/dL,abnormal
27854052,27428437,96443,103219.0,50902,2109-12-30 01:40:00,97,97.00,mEq/L,NaN
27854053,27428438,96443,103219.0,50911,2109-12-30 01:40:00,2,2.00,ng/mL,NaN


In [ ]:
microbiologyevents=read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/MICROBIOLOGYEVENTS.csv.gz')
microbiologyevents

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,SPEC_ITEMID,SPEC_TYPE_DESC,ORG_ITEMID,ORG_NAME,ISOLATE_NUM,AB_ITEMID,AB_NAME,DILUTION_TEXT,DILUTION_COMPARISON,DILUTION_VALUE,INTERPRETATION
0,744,96,170324,2156-04-13 00:00:00,2156-04-13 14:18:00,70021.0,BRONCHOALVEOLAR LAVAGE,80026.0,PSEUDOMONAS AERUGINOSA,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,745,96,170324,2156-04-20 00:00:00,2156-04-20 13:10:00,70062.0,SPUTUM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,746,96,170324,2156-04-20 00:00:00,2156-04-20 16:00:00,70012.0,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,747,96,170324,2156-04-20 00:00:00,NaN,70012.0,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,748,96,170324,2156-04-20 00:00:00,NaN,70079.0,URINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631721,630931,99783,126090,2125-11-05 00:00:00,2125-11-05 13:15:00,70076.0,TISSUE,80066.0,ASPERGILLUS FUMIGATUS,2.0,NaN,NaN,NaN,NaN,NaN,NaN
631722,630932,99783,126090,2125-11-06 00:00:00,2125-11-06 10:24:00,70076.0,TISSUE,80066.0,ASPERGILLUS FUMIGATUS,1.0,NaN,NaN,NaN,NaN,NaN,NaN
631723,630933,99783,126090,2125-11-06 00:00:00,2125-11-06 10:24:00,70076.0,TISSUE,80066.0,ASPERGILLUS FUMIGATUS,2.0,NaN,NaN,NaN,NaN,NaN,NaN
631724,630934,99783,126090,2125-11-07 00:00:00,2125-11-07 12:40:00,70012.0,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
noteevents=read_compressed_csv('/content/drive/MyDrive/mimic-iii-clinical-database-1.4/NOTEEVENTS.csv.gz')
noteevents

<ipython-input-2-27d2bf21da2b>:4: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(f)


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...
...,...,...,...,...,...,...,...,...,...,...,...
2083175,2070657,31097,115637.0,2132-01-21,2132-01-21 03:27:00,2132-01-21 03:38:00,Nursing/other,Report,17581.0,NaN,NPN\n\n\n#1 Infant remains in RA with O2 sats...
2083176,2070658,31097,115637.0,2132-01-21,2132-01-21 09:50:00,2132-01-21 09:53:00,Nursing/other,Report,19211.0,NaN,"Neonatology\nDOL #5, CGA 36 weeks.\n\nCVR: Con..."
2083177,2070659,31097,115637.0,2132-01-21,2132-01-21 16:42:00,2132-01-21 16:44:00,Nursing/other,Report,20104.0,NaN,Family Meeting Note\nFamily meeting held with ...
2083178,2070660,31097,115637.0,2132-01-21,2132-01-21 18:05:00,2132-01-21 18:16:00,Nursing/other,Report,16023.0,NaN,NPN 1800\n\n\n#1 Resp: [**Known lastname 2243*...


In [ ]:
d_icd_diagnoses_file = '/content/drive/MyDrive/mimic-iii-clinical-database-1.4/D_ICD_DIAGNOSES.csv.gz'
d_icd_diagnoses = pd.read_csv(gzip.open(d_icd_diagnoses_file, 'rb'))

d_items_file = '/content/drive/MyDrive/mimic-iii-clinical-database-1.4/D_ITEMS.csv.gz'
d_items = pd.read_csv(gzip.open(d_items_file, 'rb'))

d_icd_diagnoses_file = '/content/drive/MyDrive/mimic-iii-clinical-database-1.4/D_ICD_DIAGNOSES.csv.gz'
d_icd_diagnoses = pd.read_csv(gzip.open(d_icd_diagnoses_file, 'rb'))

In [ ]:
merged_data = patients.merge(admissions[['SUBJECT_ID', 'HADM_ID', 'HOSPITAL_EXPIRE_FLAG', 'DIAGNOSIS']], how='inner', on='SUBJECT_ID')
merged_data = merged_data.merge(icustays[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'LOS']], how='inner', on=['SUBJECT_ID', 'HADM_ID'])
merged_data = merged_data.merge(diagnoses_icd[['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE']], how='inner', on=['SUBJECT_ID', 'HADM_ID'])
merged_data = merged_data.merge(d_icd_diagnoses[['ICD9_CODE', 'SHORT_TITLE', 'LONG_TITLE']], how='inner', on='ICD9_CODE')
merged_data = merged_data.merge(labevents[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID', 'VALUE', 'VALUENUM']], how='inner', on=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'])
merged_data = merged_data.merge(d_items[['ITEMID', 'LABEL']], how='inner', on='ITEMID')
merged_data = merged_data.merge(prescriptions[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'DRUG', 'DRUG_NAME_POE']], how='inner', on=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'])

KeyError: 'ICD9_CODE'

In [ ]:
d_icd_diagnoses

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,174,01166,TB pneumonia-oth test,"Tuberculous pneumonia [any form], tubercle bac..."
1,175,01170,TB pneumothorax-unspec,"Tuberculous pneumothorax, unspecified"
2,176,01171,TB pneumothorax-no exam,"Tuberculous pneumothorax, bacteriological or h..."
3,177,01172,TB pneumothorx-exam unkn,"Tuberculous pneumothorax, bacteriological or h..."
4,178,01173,TB pneumothorax-micro dx,"Tuberculous pneumothorax, tubercle bacilli fou..."
...,...,...,...,...
14562,14432,V7399,Scrn unspcf viral dis,Special screening examination for unspecified ...
14563,14433,V740,Screening for cholera,Screening examination for cholera
14564,14434,V741,Screening-pulmonary TB,Screening examination for pulmonary tuberculosis
14565,14435,V742,Screening for leprosy,Screening examination for leprosy (Hansen's di...


In [ ]:
diagnoses_icd_file = '/content/drive/MyDrive/mimic-iii-clinical-database-1.4/DIAGNOSES_ICD.csv.gz'
diagnoses_icd = pd.read_csv(gzip.open(diagnoses_icd_file, 'rb'), dtype={'ICD9_CODE': str})

In [ ]:
diagnoses_icd

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254
...,...,...,...,...,...
651042,639798,97503,188195,2.0,20280
651043,639799,97503,188195,3.0,V5869
651044,639800,97503,188195,4.0,V1279
651045,639801,97503,188195,5.0,5275


In [ ]:
# Drop rows with missing values in ICUSTAY_ID
icustays = icustays.dropna(subset=['ICUSTAY_ID'])

# Perform a left join
merged_data = merged_data.merge(icustays[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'LOS']], how='left', on=['SUBJECT_ID', 'HADM_ID'])

# Perform a left join with labevents
merged_data = merged_data.merge(labevents[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID', 'VALUE', 'VALUENUM']], how='left', on=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'])

KeyError: "['ICUSTAY_ID'] not in index"

In [ ]:
icustays

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202
...,...,...,...,...,...,...,...,...,...,...,...,...
61527,59806,94944,143774,201233,metavision,CSRU,CSRU,15,15,2104-04-15 10:18:16,2104-04-17 14:51:00,2.1894
61528,59807,94950,123750,283653,metavision,CCU,CCU,7,7,2155-12-08 05:33:16,2155-12-10 17:24:58,2.4942
61529,59808,94953,196881,241585,metavision,SICU,SICU,57,57,2160-03-03 16:09:11,2160-03-04 14:22:33,0.9259
61530,59809,94954,118475,202802,metavision,CSRU,CSRU,15,15,2183-03-25 09:53:10,2183-03-27 17:55:03,2.3346


In [ ]:
merged_data = patients.merge(admissions[['SUBJECT_ID', 'HADM_ID', 'HOSPITAL_EXPIRE_FLAG', 'DIAGNOSIS']], how='inner', on='SUBJECT_ID')
merged_data = merged_data.merge(icustays[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'LOS']], how='inner', on=['SUBJECT_ID', 'HADM_ID'])
merged_data = merged_data.merge(diagnoses_icd[['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE']], how='inner', on=['SUBJECT_ID', 'HADM_ID'])
merged_data = merged_data.merge(d_icd_diagnoses[['ICD9_CODE', 'SHORT_TITLE', 'LONG_TITLE']], how='inner', on='ICD9_CODE')
merged_data = merged_data.merge(labevents[['SUBJECT_ID', 'HADM_ID', 'ITEMID', 'VALUE', 'VALUENUM']], how='inner', on=['SUBJECT_ID', 'HADM_ID'])
merged_data = merged_data.merge(d_items[['ITEMID', 'LABEL']], how='inner', on='ITEMID')
merged_data = merged_data.merge(prescriptions[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'DRUG', 'DRUG_NAME_POE']], how='inner', on=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'])

In [ ]:
labevents

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
0,281,3,NaN,50820,2101-10-12 16:07:00,7.39,7.39,units,NaN
1,282,3,NaN,50800,2101-10-12 18:17:00,ART,NaN,NaN,NaN
2,283,3,NaN,50802,2101-10-12 18:17:00,-1,-1.00,mEq/L,NaN
3,284,3,NaN,50804,2101-10-12 18:17:00,22,22.00,mEq/L,NaN
4,285,3,NaN,50808,2101-10-12 18:17:00,0.93,0.93,mmol/L,abnormal
...,...,...,...,...,...,...,...,...,...
27854050,27428435,96443,103219.0,50882,2109-12-30 01:40:00,26,26.00,mEq/L,NaN
27854051,27428436,96443,103219.0,50885,2109-12-30 01:40:00,2.1,2.10,mg/dL,abnormal
27854052,27428437,96443,103219.0,50902,2109-12-30 01:40:00,97,97.00,mEq/L,NaN
27854053,27428438,96443,103219.0,50911,2109-12-30 01:40:00,2,2.00,ng/mL,NaN


##Efficent way to handle larger sized files:

In [ ]:
import dask.dataframe as dd

In [ ]:
patients_file = '/content/drive/MyDrive/mimic-iii-clinical-database-1.4/PATIENTS.csv.gz'
patients = dd.read_csv(patients_file, blocksize=None, dtype={'SUBJECT_ID': str})

admissions_file = '/content/drive/MyDrive/mimic-iii-clinical-database-1.4/ADMISSIONS.csv'
admissions = dd.read_csv(admissions_file, blocksize=None, dtype={'SUBJECT_ID': str})

diagnoses_icd_file = '/content/drive/MyDrive/mimic-iii-clinical-database-1.4/DIAGNOSES_ICD.csv.gz'
diagnoses_icd = dd.read_csv(diagnoses_icd_file, blocksize=None, dtype={'SUBJECT_ID': str, 'ICD9_CODE': str})

noteevents_file = '/content/drive/MyDrive/mimic-iii-clinical-database-1.4/NOTEEVENTS.csv.gz'
noteevents = dd.read_csv(noteevents_file, blocksize=None, dtype={'SUBJECT_ID': str})

In [ ]:
merged_data = patients.merge(admissions[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DIAGNOSIS']], how='inner', on='SUBJECT_ID')
merged_data = merged_data.merge(diagnoses_icd[['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE']], how='inner', on=['SUBJECT_ID', 'HADM_ID'])
merged_data = merged_data.merge(noteevents[['SUBJECT_ID', 'HADM_ID', 'TEXT', 'CATEGORY', 'DESCRIPTION']], how='inner', on=['SUBJECT_ID', 'HADM_ID'])

In [ ]:
output_file = '/content/drive/MyDrive/mimic-iii-clinical-database-1.4/Merged_files/patient_admission_diagnoses_noteevents.csv'

chunk_size = 10000  # Define your desired chunk size
merged_data.to_csv(output_file, index=False, single_file=True, chunksize=chunk_size)


In [ ]:
merged_data = patients.merge(admissions[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DIAGNOSIS']], how='inner', on='SUBJECT_ID')
merged_data = merged_data.merge(diagnoses_icd[['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE']], how='inner', on=['SUBJECT_ID', 'HADM_ID'])
merged_data = merged_data.merge(noteevents[['SUBJECT_ID', 'HADM_ID', 'TEXT', 'CATEGORY', 'DESCRIPTION']], how='inner', on=['SUBJECT_ID', 'HADM_ID'])